In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm  #progress bar
import plotly.graph_objects as go
import plotly.express as px
import plotly as py
import random
from collections import Counter #개수 세기 모듈
from ast import literal_eval

In [ ]:
import nltk #자연어 처리 패키지
from nltk.corpus import stopwords #불용어 모듈
nltk.download('all') #nltk 기능 다운로드 (1분)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **원본 arxiv CS. 데이터(분석에 사용하지 않음)**

In [ ]:
#분류 전 데이터
file_list=sorted(glob('/content/drive/MyDrive/Colab Notebooks/EDA 3조(code)/arxiv.csv'))
data=pd.DataFrame()
for file in tqdm(file_list):
    temp = pd.read_csv(file, encoding='utf-8')
    data = pd.concat([data, temp], axis=0)

## **대략적인 정보 확인**

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438727 entries, 0 to 438726
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    438727 non-null  int64 
 1   id            438727 non-null  object
 2   authors       438727 non-null  object
 3   title         438727 non-null  object
 4   categories    438727 non-null  object
 5   created       438727 non-null  object
 6   last_updated  438727 non-null  object
 7   submitter     438599 non-null  object
 8   comments      268362 non-null  object
 9   journal-ref   56649 non-null   object
 10  doi           73663 non-null   object
 11  report-no     7582 non-null    object
 12  license       429524 non-null  object
 13  abstract      438727 non-null  object
dtypes: int64(1), object(13)
memory usage: 46.9+ MB


## **Column 정제**

In [ ]:
#번호 열 삭제
#data=data.drop(columns='Unnamed: 0',axis=1)
data.head(1)

,Unnamed: 0,id,authors,title,categories,created,last_updated,submitter,comments,journal-ref,doi,report-no,license,abstract
0,46,704.0047,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,"Sun, 1 Apr 2007 13:06:50 GMT","Sun, 1 Apr 2007 13:06:50 GMT",Igor Grabec,"5 pages, 5 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,The intelligent acoustic emission locator is...


In [ ]:
#날짜 오브젝트로 변환(2분)
data['created']=pd.to_datetime(data['created'])
data['last_updated']=pd.to_datetime(data['last_updated'])

KeyboardInterrupt: ignored

In [ ]:
data.info()

In [ ]:
data.head()

## **Uniqueness 확인**

In [ ]:
#작성자의 uniqueness (1에 가까울수록 특정성이 없음)
len(data['authors'].unique())/len(data)

0.8912581172346357

In [ ]:
#제출자의 uniqueness
len(data['submitter'].unique())/len(data[data['submitter'].notna()])

0.35224658514953294

In [ ]:
#개제된 장소의 uniqueness
len(data['journal-ref'].unique())/len(data[data['journal-ref'].notna()])

0.9309255238397853

## **제목 열 토큰화**

In [ ]:
#제목 열 추출
titles = data.title
titles

0         Intelligent location of simultaneously active ...
1         Intelligent location of simultaneously active ...
2         On-line Viterbi Algorithm and Its Relationship...
3                 Real Options for Project Schedules (ROPS)
4         Sparsely-spread CDMA - a statistical mechanics...
                                ...                        
438722    Mixed-Level Knowledge Representation and Varia...
438723    Computation in an algebra of test selection cr...
438724                     Quantum Bit Commitment Expansion
438725                   Additive models in high dimensions
438726    Seeing the Forest in the Tree: Applying VRML t...
Name: title, Length: 438727, dtype: object

In [ ]:
import nltk #자연어 처리 패키지
from nltk.corpus import stopwords #불용어 모듈

In [ ]:
nltk.download('all') #nltk 기능 다운로드 (3분)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
tokenized_titles = [nltk.word_tokenize(i) for i in titles] #제목을 토큰화(1분)
tokenized_titles

In [ ]:
from collections import Counter #개수 세기 모듈

In [ ]:
#제목 토큰을 1차원 리스트로
one_di_corpus=[]
for i in tokenized_titles:
    for j in i:
        one_di_corpus.append(j)
one_di_corpus

In [ ]:
counter=Counter(one_di_corpus)
counter.most_common(100)

[('for', 159121),
 ('of', 120880),
 (':', 117120),
 ('and', 106764),
 ('in', 80369),
 ('with', 59157),
 ('A', 58792),
 ('the', 55138),
 ('Learning', 53813),
 ('on', 33274),
 ('to', 31162),
 ('a', 28589),
 ('Networks', 28204),
 (',', 23555),
 ('Neural', 21671),
 ('Deep', 19582),
 ('using', 17489),
 ('Data', 17303),
 ('Network', 17087),
 ('Detection', 16263),
 ('via', 15525),
 ('from', 14936),
 ('An', 13982),
 ('Using', 13944),
 ('The', 13683),
 ('Analysis', 13431),
 ('Systems', 12914),
 ('Models', 12756),
 ('Model', 12260),
 ('On', 12200),
 ('by', 10871),
 ('Image', 10711),
 ('Graph', 10270),
 ('Approach', 9326),
 ('$', 9114),
 ('Efficient', 9034),
 ('Language', 8839),
 ('(', 8706),
 (')', 8705),
 ('Classification', 8371),
 ('Based', 8291),
 ('Recognition', 8270),
 ('based', 7991),
 ('?', 7818),
 ('Machine', 7746),
 ('Reinforcement', 7695),
 ('Framework', 7616),
 ('Towards', 7596),
 ('Optimization', 7344),
 ('Information', 7332),
 ('System', 7279),
 ('Estimation', 7223),
 ('Adversarial'

In [ ]:
#불용어 리스트
stop_words=stopwords.words('english')

In [ ]:
#그 외 불용어 리스트
punc_marks=[',',':','?','(',')','$','\'s','--','-']

In [ ]:
#소문자로 바꾸기
titles_low=[]
for i in  one_di_corpus:
  titles_low.append(i.lower())

In [ ]:
stops=stop_words+punc_marks # 두 리스트를 하나로

In [ ]:
#불용어 제거
titles_eli = []
for i in titles_low:
    if i not in stops:
        titles_eli.append(i)
titles_eli

In [ ]:
counter=Counter(titles_eli)
counter.most_common(100)

In [ ]:
#불용어 제거
titles_cap_eli = []
for i in one_di_corpus:
    if i not in stops:
        titles_eli.append(i)
titles_cap_eli

[]

In [ ]:
counter=Counter(titles_cap_eli)
counter.most_common(100)

[]

In [ ]:
data['license'].unique()

array([nan, 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
       'http://creativecommons.org/licenses/publicdomain/',
       'http://creativecommons.org/licenses/by-nc-sa/3.0/',
       'http://creativecommons.org/licenses/by/4.0/',
       'http://creativecommons.org/licenses/by/3.0/',
       'http://creativecommons.org/licenses/by-nc-sa/4.0/',
       'http://creativecommons.org/licenses/by-nc-nd/4.0/',
       'http://creativecommons.org/licenses/by-sa/4.0/',
       'http://creativecommons.org/publicdomain/zero/1.0/'], dtype=object)

## **arxiv에서 AI 관련 카테고리만 뽑기**

- AI(AI의 전반적인 논문)
- CL(자연어처리관련 논문)
- CV(컴퓨터 비전관련 논문)
- FL(Formal Languages and Automata Theory관련 논문)
- LG(ML 관련 논문)
- NE(Neural and Evolutionary Computing관련 논문)

In [ ]:
#년월 칼럼 추가
data['created_ym']=data['created'].dt.strftime('%Y-%m')

In [ ]:
#date오브젝트로
data['created_ym']=pd.to_datetime(data['created_ym'])

In [ ]:
data.head()

,id,authors,title,categories,created,last_updated,submitter,comments,journal-ref,doi,report-no,license,abstract,created_ym
0,704.0047,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,2007-04-01 13:06:50+00:00,2007-04-01 13:06:50+00:00,Igor Grabec,"5 pages, 5 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,The intelligent acoustic emission locator is...,2007-04-01
1,704.005,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,2007-04-01 18:53:13+00:00,2007-04-01 18:53:13+00:00,Igor Grabec,"5 pages, 7 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,Part I describes an intelligent acoustic emi...,2007-04-01
2,704.0062,"Rastislav \v{S}r\'amek, Bro\v{n}a Brejov\'a, T...",On-line Viterbi Algorithm and Its Relationship...,cs.DS,2007-03-31 23:52:33+00:00,2007-03-31 23:52:33+00:00,Tom\'a\v{s} Vina\v{r},NaN,Algorithms in Bioinformatics: 7th Internationa...,10.1007/978-3-540-74126-8_23,NaN,NaN,"In this paper, we introduce the on-line Vite...",2007-03-01
3,704.009,Lester Ingber,Real Options for Project Schedules (ROPS),cs.CE cond-mat.stat-mech cs.MS cs.NA physics.d...,2007-04-01 14:35:40+00:00,2007-04-01 14:35:40+00:00,Lester Ingber,NaN,NaN,NaN,Report 2007:ROPS,NaN,Real Options for Project Schedules (ROPS) ha...,2007-04-01
4,704.0098,"Jack Raymond, David Saad",Sparsely-spread CDMA - a statistical mechanics...,cs.IT math.IT,2007-04-01 18:27:26+00:00,2008-04-30 15:36:55+00:00,Jack Raymond,"23 pages, 5 figures, figure 1 amended since pu...",J. Phys. A: Math. Theor. 40 No 41 (12 October ...,10.1088/1751-8113/40/41/004,NaN,NaN,"Sparse Code Division Multiple Access (CDMA),...",2007-04-01


In [ ]:
data_all = data[data['categories'].str.contains("cs.AI|cs.CL|cs.CV|cs.FL|cs.LG|cs.NE")]

In [ ]:
data_all.head()

,id,authors,title,categories,created,last_updated,submitter,comments,journal-ref,doi,report-no,license,abstract,created_ym
0,704.0047,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,2007-04-01 13:06:50+00:00,2007-04-01 13:06:50+00:00,Igor Grabec,"5 pages, 5 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,The intelligent acoustic emission locator is...,2007-04-01
1,704.005,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,2007-04-01 18:53:13+00:00,2007-04-01 18:53:13+00:00,Igor Grabec,"5 pages, 7 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,Part I describes an intelligent acoustic emi...,2007-04-01
12,704.0304,Carlos Gershenson,The World as Evolving Information,cs.IT cs.AI math.IT q-bio.PE,2007-04-03 02:08:48+00:00,2010-10-13 19:49:16+00:00,Carlos Gershenson,"16 pages. Extended version, three more laws of...","Minai, A., Braha, D., and Bar-Yam, Y., eds. Un...",10.1007/978-3-642-18003-3_10,NaN,http://arxiv.org/licenses/nonexclusive-distrib...,This paper discusses the benefits of describ...,2007-04-01
21,704.0671,Maxim Raginsky,Learning from compressed observations,cs.IT cs.LG math.IT,2007-04-05 02:57:15+00:00,2007-04-05 02:57:15+00:00,Maxim Raginsky,6 pages; submitted to the 2007 IEEE Informatio...,NaN,10.1109/ITW.2007.4313111,NaN,NaN,The problem of statistical learning is to co...,2007-04-01
34,704.0954,Soummya Kar and Jose M. F. Moura,Sensor Networks with Random Links: Topology De...,cs.IT cs.LG math.IT,2007-04-06 21:58:52+00:00,2007-04-06 21:58:52+00:00,Jos\'e M. F. Moura,Submitted to IEEE Transactions,NaN,10.1109/TSP.2008.920143,NaN,NaN,"In a sensor network, in practice, the commun...",2007-04-01


In [ ]:
import os

#파일로 저장
data_all.to_csv('arxiv_artin.csv')

NameError: ignored

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm  #progress bar

In [ ]:
#파일 불러오기

file_list=sorted(glob('/content/drive/MyDrive/Colab Notebooks/EDA 3조(code)/arxiv_ai.csv'))


In [ ]:
data=pd.DataFrame()
for file in tqdm(file_list):
    temp = pd.read_csv(file, encoding='utf-8')
    data = pd.concat([data, temp], axis=0)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data.head()

""


In [ ]:
#카테고리로 분류

ctg_name={'cs.AI':data_ai,'cs.CL':data_cl,'cs.FL':data_fl,'cs.CV':data_cv,'cs.LG':data_lg,'cs.NE':data_ne} #변수와 이름 딕셔너리

for i in ctg_name:
  ctg_name[i]=data[data['categories'].str.contains(i)]

SyntaxError: ignored

In [ ]:
#카테고리 열 추가
for i in ctg_name:
  ctg_name[i]['category']=i

NameError: ignored

In [ ]:
data_ai.head()

AttributeError: ignored

In [ ]:
#다시 하나로
list_ctg=[]
for i in ctg_name:
  list_ctg.append(ctg_name[i])

data_all=pd.concat(list_ctg)

In [ ]:
data_all

,Unnamed: 0,id,authors,title,categories,created,last_updated,submitter,comments,journal-ref,doi,report-no,license,abstract,created_ym,category
0,0,704.0047,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,2007-04-01 13:06:50+00:00,2007-04-01 13:06:50+00:00,Igor Grabec,"5 pages, 5 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,The intelligent acoustic emission locator is...,2007-04-01,cs.AI
1,1,704.005,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,2007-04-01 18:53:13+00:00,2007-04-01 18:53:13+00:00,Igor Grabec,"5 pages, 7 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,Part I describes an intelligent acoustic emi...,2007-04-01,cs.AI
2,12,704.0304,Carlos Gershenson,The World as Evolving Information,cs.IT cs.AI math.IT q-bio.PE,2007-04-03 02:08:48+00:00,2010-10-13 19:49:16+00:00,Carlos Gershenson,"16 pages. Extended version, three more laws of...","Minai, A., Braha, D., and Bar-Yam, Y., eds. Un...",10.1007/978-3-642-18003-3_10,NaN,http://arxiv.org/licenses/nonexclusive-distrib...,This paper discusses the benefits of describ...,2007-04-01,cs.AI
3,36,704.0985,"Mohd Abubakr, R.M.Vinay",Architecture for Pseudo Acausal Evolvable Embe...,cs.NE cs.AI,2007-04-07 13:40:49+00:00,2007-04-07 13:40:49+00:00,Mohd Abubakr,"4 pages, 2 figures. Submitted to SASO 2007",NaN,NaN,NaN,NaN,Advances in semiconductor technology are con...,2007-04-01,cs.AI
4,38,704.1028,Jianlin Cheng,A neural network approach to ordinal regression,cs.LG cs.AI cs.NE,2007-04-08 17:36:00+00:00,2007-04-08 17:36:00+00:00,Jianlin Cheng,8 pages,NaN,NaN,NaN,NaN,Ordinal regression is an important type of l...,2007-04-01,cs.AI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333585,438471,cs/9902006,Paul Vitanyi,A Discipline of Evolutionary Programming,cs.NE cs.AI cs.CC cs.DS cs.LG cs.MA,1999-02-02 16:17:16+00:00,1999-02-02 16:17:16+00:00,Paul Vitanyi,"25 pages, LaTeX source, Theoretical Computer S...","Theoret. Comp. Sci., 241:1-2 (2000), 3--23.",NaN,NaN,NaN,Genetic fitness optimization using small pop...,1999-02-01,cs.NE
333586,438490,cs/9902025,"Mattias Ohlsson, Carsten Peterson and Bo S\""od...",An Efficient Mean Field Approach to the Set Co...,cs.NE,1999-02-12 09:18:53+00:00,1999-02-12 09:18:53+00:00,Mattias Ohlsson,"17 pages, 2 figures",NaN,NaN,NaN,NaN,A mean field feedback artificial neural netw...,1999-02-01,cs.NE
333587,438547,cs/9905011,"Kagan Tumer, Nirmala Ramanujam, Joydeep Ghosh,...",Ensembles of Radial Basis Function Networks fo...,cs.NE cs.LG q-bio,1999-05-20 18:28:15+00:00,1999-05-20 18:28:15+00:00,Kagan Tumer,23 pages,"IEEE Transactions on Biomedical Engineering, v...",NaN,NaN,NaN,The mortality related to cervical cancer can...,1999-05-01,cs.NE
333588,438548,cs/9905012,Kagan Tumer and Joydeep Ghosh,Linear and Order Statistics Combiners for Patt...,cs.NE cs.LG,1999-05-20 20:15:13+00:00,1999-05-20 20:15:13+00:00,Kagan Tumer,31 pages,"Combining Artificial Neural Networks,Ed. Amand...",NaN,NaN,NaN,Several researchers have experimentally show...,1999-05-01,cs.NE


## **시각화**

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly as py

**시간에 따른 전체 AI 논문 발행 수**

In [ ]:
data_ctg=data_all[['id','created_ym']].groupby('created_ym').count().rename(columns={'id': 'count'})
data_ctg['date']=data_ctg.index
data_ctg

,count,date
created_ym,,
1993-08-01,2,1993-08-01
1993-09-01,1,1993-09-01
1993-11-01,2,1993-11-01
1993-12-01,1,1993-12-01
1994-01-01,1,1994-01-01
...,...,...
2023-04-01,9601,2023-04-01
2023-05-01,16683,2023-05-01
2023-06-01,12737,2023-06-01


In [ ]:
fig = px.line(data_ctg, x='date', y='count', markers=False)
fig.show()

NameError: ignored

**분야별 시간에 따른 논문 발행 수**

In [ ]:
#각 카테고리 별로 연월 별 개수 그룹화
ctg_name={'cs.AI':[data_ai,data_ai_ctg],'cs.CL':[data_cl,data_cl_ctg],'cs.FL':[data_fl,data_fl_ctg],'cs.CV':[data_cv,data_cv_ctg],'cs.LG':[data_lg,data_lg_ctg],'cs.NE':[data_ne,data_ne_ctg]} #변수와 이름 딕셔너리

for i in ctg_name:
  ctg_name[i][1]=ctg_name[i][0][['id','created_ym']].groupby('created_ym').count().rename(columns={'id': 'count'})
  ctg_name[i][1]['category']=i #카테고리 열 생성
  ctg_name[i][1]['date']=ctg_name[i][1].index #날짜 인덱스를 칼럼으로
data_ne_ctg

,counts,category,date
created_ym,,,
1998-09-01,5,NE,1998-09-01
1998-11-01,3,NE,1998-11-01
1998-12-01,4,NE,1998-12-01
1999-02-01,4,NE,1999-02-01
1999-05-01,7,NE,1999-05-01
...,...,...,...
2023-04-01,276,NE,2023-04-01
2023-05-01,323,NE,2023-05-01
2023-06-01,227,NE,2023-06-01


In [ ]:
#하나로 합치기
data_date_list=[]
for i in ctg_name:
  data_date_list.append(ctg_name[i][1])
data_date=pd.concat(data_date_list)
data_date

,count,category,date
created_ym,,,
1993-08-01,2,cs.AI,1993-08-01
1993-09-01,1,cs.AI,1993-09-01
1993-11-01,2,cs.AI,1993-11-01
1993-12-01,1,cs.AI,1993-12-01
1994-01-01,1,cs.AI,1994-01-01
...,...,...,...
2023-04-01,276,cs.NE,2023-04-01
2023-05-01,323,cs.NE,2023-05-01
2023-06-01,227,cs.NE,2023-06-01


In [ ]:
fig = px.line(data_date, x='date', y='count', color='category', markers=False, title='카테고리별 시간에 따른 논문 발행 수')


fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
pip install chart_studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.3 MB/s eta 0:00:00


**(웹으로 보내기)**

In [ ]:
import chart_studio.tools #plotly 연동 모듈
username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
import chart_studio.plotly as py
chart_studio.plotly.plot(fig, filename = '분야 별 총 논문 발행 수 비교', auto_open=True)

'https://plotly.com/~biniy/8/'

**카테고리별 총 논문 발행 수 비교 boxplot**

In [ ]:
data_date

,count,category,date
created_ym,,,
1993-08-01,2,cs.AI,1993-08-01
1993-09-01,1,cs.AI,1993-09-01
1993-11-01,2,cs.AI,1993-11-01
1993-12-01,1,cs.AI,1993-12-01
1994-01-01,1,cs.AI,1994-01-01
...,...,...,...
2023-04-01,276,cs.NE,2023-04-01
2023-05-01,323,cs.NE,2023-05-01
2023-06-01,227,cs.NE,2023-06-01


In [ ]:
fig = px.bar(data_date, x='category', y='count', color="date", text='date',
             title="분야별 총 논문 발행 수 비교(가데이터. 향후 color=중분류)")
fig.update_traces(textposition='inside')

#타임스크롤 넣을 예정
fig.show()

**각 분야 내의 인기 모델(분야별로 각각 그래프)**

In [ ]:
#드롭다운 메뉴+한 페이지에 분야를 모두 모아 나열로 구성 예정


**카테고리 별 워드 클라우드(시간별)**

In [ ]:
#pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3.7


**동적 워드 클라우드 시도 중...(plotly에 없음)**

In [ ]:
import plotly.graph_objects as go
from dash import dcc
import random

ModuleNotFoundError: ignored

In [ ]:
import nltk #자연어 처리 패키지
from nltk.corpus import stopwords #불용어 모듈

In [ ]:
tokenized_titles = [nltk.word_tokenize(i) for i in titles] #제목을 토큰화(1분)
tokenized_titles

NameError: ignored

In [ ]:
from collections import Counter #개수 세기 모듈

In [ ]:
#제목 토큰을 1차원 리스트로
one_di_corpus=[]
for i in tokenized_titles:
    for j in i:
        one_di_corpus.append(j)
one_di_corpus

NameError: ignored

### word list

In [ ]:
word_list=['Intelligent',
 'location',
 'of',
 'simultaneously',
 'active',
 'acoustic',
 'emission',
 'sources',
 ':',
 'Part',
 'I',
 'Intelligent',
 'location',
 'of',
 'simultaneously',
 'active',
 'acoustic',
 'emission',
 'sources',
 ':',
 'Part',
 'II',
 'On-line',
 'Viterbi',
 'Algorithm',
 'and',
 'Its',
 'Relationship',
 'to',
 'Random',
 'Walks',
 'Real',
 'Options',
 'for',
 'Project',
 'Schedules',
 '(',
 'ROPS',
 ')',
 'Sparsely-spread',
 'CDMA',
 '-',
 'a',
 'statistical',
 'On',
 'Almost',
 'Periodicity',
 'Criteria',
 'for',
 'Morphic',
 'Sequences',
 'in',
 'Some',
 'Particular',
 'Cases',
 'Geometric',
 'Complexity',
 'Theory',
 'VI',
 ':',
 'the',
 'flip',
 'via',
 'saturated',
 'and',
 'positive',
 'integer',
 'programming',
 'in',
 'representation',
 'theory',
 'and',
 'algebraic',
 'geometry',
 'On',
 'Punctured',
 'Pragmatic',
 'Space-Time',
 'Codes',
 'in',
 'Block',
 'Fading',
 'Channel',
 'Differential',
 'Recursion',
 'and',
 'Differentially',
 'Algebraic',
 'Functions',
 'The',
 'World',
 'as',
 'Evolving',
 'Information',
 'The',
 'Complexity',
 'of',
 'HCP',
 'in',
 'Digraps',
 'with',
 'Degree',
 'Bound',
 'Two',
 'Pseudo-random',
 'Puncturing',
 ':',
 'A',
 'Technique',
 'to',
 'Lower',
 'the',
 'Error',
 'Floor',
 'of',
 'Turbo',
 'Codes',
 'Inapproximability',
 'of',
 'Maximum',
 'Weighted',
 'Edge',
 'Biclique',
 'and',
 'Its',
 'Applications',
 'Refuting',
 'the',
 'Pseudo',
 'Attack',
 'on',
 'the',
 'REESSE1+',
 'Cryptosystem',
 'Optimal',
 'Routing',
 'for',
 'Decode-and-Forward',
 'based',
 'Cooperation',
 'in',
 'Wireless',
 'Networks',
 'Many-to-One',
 'Throughput',
 'Capacity',
 'of',
 'IEEE',
 '802.11',
 'Multi-hop',
 'Wireless',
 'Networks',
 'On',
 'the',
 'Achievable',
 'Rate',
 'Regions',
 'for',
 'Interference',
 'Channels',
 'with',
 'Degraded',
 'Message',
 'Sets',
 'A',
 'Low',
 'Complexity',
 'Algorithm',
 'and',
 'Architecture',
 'for',
 'Systematic',
 'Encoding',
 'of',
 'Hermitian',
 'Codes',
 'Learning',
 'from',
 'compressed',
 'observations',
 'Revisiting',
 'the',
 'Issues',
 'On',
 'Netflow',
 'Sample',
 'and',
 'Export',
 'Performance',
 'Optimal',
 'Synthesis',
 'of',
 'Multiple',
 'Algorithms',
 'Hybrid-ARQ',
 'in',
 'Multihop',
 'Networks',
 'with',
 'Opportunistic',
 'Relay',
 'Selection',
 'Opportunistic',
 'Relay',
 'Selection',
 'with',
 'Limited',
 'Feedback',
 'On',
 'packet',
 'lengths',
 'and',
 'overhead',
 'for',
 'random',
 'linear',
 'coding',
 'over',
 'the',
 'erasure',
 'channel',
 'P-adic',
 'arithmetic',
 'coding',
 'Universal',
 'Source',
 'Coding',
 'for',
 'Monotonic',
 'and',
 'Fast',
 'Decaying',
 'Monotonic',
 'Distributions',
 'Lessons',
 'Learned',
 'from',
 'the',
 'deployment',
 'of',
 'a',
 'high-interaction',
 'honeypot',
 'Availability',
 'assessment',
 'of',
 'SunOS/Solaris',
 'Unix',
 'Systems',
 'based',
 'on',
 'Syslogd',
 'and',
 'wtmpx',
 'logfiles',
 ':',
 'a',
 'case',
 'study',
 'Empirical',
 'analysis',
 'and',
 'statistical',
 'modeling',
 'of',
 'attack',
 'processes',
 'based',
 'on',
 'honeypots',
 'An',
 'architecture-based',
 'dependability',
 'modeling',
 'framework',
 'using',
 'AADL',
 'A',
 'Hierarchical',
 'Approach',
 'for',
 'Dependability',
 'Analysis',
 'of',
 'a',
 'Commercial',
 'Cache-Based',
 'RAID',
 'Storage',
 'Architecture',
 'Sensor',
 'Networks',
 'with',
 'Random',
 'Links',
 ':',
 'Topology',
 'Design',
 'for',
 'Distributed',
 'Consensus',
 'Cross-Layer',
 'Optimization',
 'of',
 'MIMO-Based',
 'Mesh',
 'Networks',
 'with',
 'Gaussian',
 'Vector',
 'Broadcast',
 'Channels',
 'Architecture',
 'for',
 'Pseudo',
 'Acausal',
 'Evolvable',
 'Embedded',
 'Systems',
 'The',
 'on-line',
 'shortest',]


### 시작

In [ ]:
counter=Counter(word_list)
words=dict(counter)
words

NameError: ignored

In [ ]:
word_count=len(words)
import random
import pandas as pd

In [ ]:
x=random.sample(range(1,500),219)
y=random.sample(range(1,500),219)
freq_list=[words[i] for i in words]

In [ ]:
list(words.keys())

In [ ]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import random

In [ ]:
#컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯) - opacity 조절 시도해보기(아마 안 될듯...)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)

In [ ]:
color_list=[f'hsv({ran_hue(9,38)},{ran_per(0,100)}%,{ran_per(30,80)}%)' for i in range(word_count+1)]

In [ ]:
opa=[random.uniform(0,1) for i in range(word_count+1)]

In [ ]:

# get the relative occurence frequencies
new_freq_list = []
for i in freq_list:
    new_freq_list.append(i*10)
new_freq_list


#흩뿌려지는 범위 고민하기(글자가 잘림)
trace = go.Scatter(x=x,
                    y=y,
                    textfont = {'size':new_freq_list,
                                    'color':color_list},
                    hoverinfo=freq_list, #이건 전체에 대한 비율을 보여주는 것도 좋을 듯
                    mode="text",
                    text=list(words.keys()),
                  )

py.graph_objects.scatter.Marker(opacity=0.5)
layout = go.Layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True)
                  )
fig = go.Figure(data=[trace], layout=layout)
fig.show()



In [ ]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import random
words = words = ['征信', '拍拍贷', '查询', '报告', '贷款', '个人', '怎么', '信用卡', '逾期', '被拒', '如何', '中心', '信用', '网贷', '人人', '分期', '注册', '好信', '手机', '钱包', '个人信用', '借呗', '平安', '捷信', '微粒贷', '借钱', '记录', '用钱', '可以', '花呗', '身份证', '拍拍', '现金', '微信', '还款', '问问', '产品', '51', '信而富', '什么', '黑名单', '360', '17', '黑户', '怎么办', '金融', '帮你贷', '消除', '密码', '账号', '怎样', '分期乐', '拒绝', '申请']


frequency = [1083, 393, 353, 167, 123, 119, 83, 64, 57, 46, 44, 40, 37, 31, 29, 29, 28, 26, 25, 23, 23, 22, 21, 19, 18, 18, 18, 18, 18, 17, 15, 15, 15, 15, 14, 14, 13, 13, 13, 13, 13, 13, 12, 12, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10]

lower, upper = 15, 45
frequency = [((x - min(frequency)) / (max(frequency) - min(frequency))) * (upper - lower) + lower for x in frequency]


percent = [0.362086258776329, 0.13139418254764293, 0.11802072885322636, 0.055834169174189235, 0.041123370110330994, 0.03978602474088933, 0.02774991641591441, 0.02139752591106653, 0.01905717151454363, 0.015379471748579069, 0.01471079906385824, 0.013373453694416584, 0.012370444667335341, 0.010364426613172852, 0.009695753928452023, 0.009695753928452023, 0.009361417586091608, 0.008692744901370779, 0.008358408559010365, 0.0076897358742895345, 0.0076897358742895345, 0.00735539953192912, 0.007021063189568706, 0.006352390504847877, 0.006018054162487462, 0.006018054162487462, 0.006018054162487462, 0.006018054162487462, 0.006018054162487462, 0.0056837178201270475, 0.005015045135406218, 0.005015045135406218, 0.005015045135406218, 0.005015045135406218, 0.004680708793045804, 0.004680708793045804, 0.0043463724506853894, 0.0043463724506853894, 0.0043463724506853894, 0.0043463724506853894, 0.0043463724506853894, 0.0043463724506853894, 0.004012036108324975, 0.004012036108324975, 0.00367769976596456, 0.00367769976596456, 0.00367769976596456, 0.00367769976596456, 0.003343363423604146, 0.003343363423604146, 0.003343363423604146, 0.003343363423604146, 0.003343363423604146, 0.003343363423604146]

lenth = len(words)
colors = [py.colors.DEFAULT_PLOTLY_COLORS[random.randrange(1, 10)] for i in range(lenth)]

data = go.Scatter(
  x=list(range(lenth)),
  y=random.choices(range(lenth), k=lenth),
  mode='text',
  text=words,
  hovertext=['{0}{1}{2}'.format(w, f, format(p, '.2%')) for w, f, p in zip(words, frequency, percent)],
  hoverinfo='text',
  textfont={'size': frequency, 'color': colors})
layout = go.Layout({'xaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False},
                    'yaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False}})

fig = go.Figure(data=[data], layout=layout)

fig.show()

# **arxiv CS. 하위 카테고리에서 AI 관련  카테고리만 가져온 데이터 (실사용 데이터1)**

AI(AI의 전반적인 논문)  
CL(자연어처리관련 논문)  
CV(컴퓨터 비전관련 논문)  
FL(Formal Languages and Automata Theory관련 논문)  
LG(ML 관련 논문)  
NE(Neural and Evolutionary Computing관련 논문)  

## **전처리**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#이 데이터는 Computer Science카테고리만 있는 데이터임
file_list=sorted(glob('/content/drive/MyDrive/Colab Notebooks/EDA 3조(code)/arxiv_cs.csv'))


In [ ]:
data=pd.DataFrame()
for file in tqdm(file_list):
    temp = pd.read_csv(file, encoding='utf-8')
    data = pd.concat([data, temp], axis=0)

  0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-10-0a76881623e2>:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(file, encoding='utf-8')


In [ ]:
#날짜 오브젝트로 변환(2분 20초)
data['created']=pd.to_datetime(data['created'])
data['last_updated']=pd.to_datetime(data['last_updated'])

In [ ]:
#연월 칼럼 추가
data['created_ym']=data['created'].dt.strftime('%Y-%m')

In [ ]:
data.head(1)

,Unnamed: 0,id,authors,title,categories,created,last_updated,submitter,comments,journal-ref,doi,report-no,license,abstract,created_ym
0,46,704.0047,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,2007-04-01 13:06:50+00:00,2007-04-01 13:06:50+00:00,Igor Grabec,"5 pages, 5 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,The intelligent acoustic emission locator is...,2007-04


In [ ]:
#연월 별 개수 그룹화
data_all=data[['id','created_ym']].groupby('created_ym').count().rename(columns={'id': 'count'})

In [ ]:
data_all['date']=data_all.index

## **시간 별 총 논문 발행 수 그래프**

In [ ]:
fig = px.bar(data_all, x='date', y='count', title="시간에 따른 논문 발행 수(분류 전 데이터)")
fig.update_traces(textposition="outside")
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

NameError: ignored

# **method를 통해 분류된 데이터(실사용 데이터2)**

## **데이터 전처리**

In [ ]:
file_list=sorted(glob('/content/drive/MyDrive/Colab Notebooks/EDA 3조(code)/치웅님 서치/data_ai_fin2.csv'))
data=pd.DataFrame()
for file in tqdm(file_list):
    temp = pd.read_csv(file, encoding='utf-8')
    data = pd.concat([data, temp], axis=0)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame


In [ ]:
#컬럼 전처리(날짜 오브젝트로)
data['created_datetime']=pd.to_datetime(data['created_datetime'])
data['last_updated_datetime']=pd.to_datetime(data['last_updated_datetime'])
data['created_datetime_month']=pd.to_datetime(data['created_datetime_month'])

In [ ]:
data.info()

In [ ]:
data.head(1)

,ori_index,id,authors,title,categories,submitter,comments,journal-ref,doi,report-no,...,abstract,created_datetime,last_updated_datetime,created_datetime_month,is_CV,is_NLP,is_RL,is_Au,is_Se,is_Gr
0,46,704.0047,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,Igor Grabec,"5 pages, 5 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,...,The intelligent acoustic emission locator is...,2007-04-01,2007-04-01,2007-04-01,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#분류가 된 것의 비율
len(data[data[['is_CV','is_NLP','is_RL','is_Au','is_Se','is_Gr']].notnull().any(axis=1)])/len(data)*100

53.94027220417436

In [ ]:
#분류된 데이터만 모으기
data_cla=data[data[['is_CV','is_NLP','is_RL','is_Au','is_Se','is_Gr']].notnull().any(axis=1)]

In [ ]:
#분야별 데이터 프레임 만들기
cla_list=['is_CV','is_NLP','is_RL','is_Au','is_Se','is_Gr']
for i in cla_list:
  globals()[i]=data_cla[data_cla[i].notnull()]

In [ ]:
is_NLP.info()

In [ ]:
#카테고리 컬럼 추가 및 기존 카테고리 칼럼 제거
cla_list_name={'is_CV':[is_CV,'CV'],'is_NLP':[is_NLP,'NLP'],'is_RL':[is_RL,'RL'],'is_Au':[is_Au,'Audio'],'is_Se':[is_Se,'Sequential'],'is_Gr':[is_Gr,'Graph']}
for i in cla_list_name:
  cla_list_name[i][0]['category']=cla_list_name[i][1]
  cla_list_name[i][0].drop(columns=cla_list,axis=1, inplace=True)

In [ ]:
is_CV.head(1)

,ori_index,id,authors,title,categories,submitter,comments,journal-ref,doi,report-no,license,abstract,created_datetime,last_updated_datetime,created_datetime_month,category
10,2009,704.201,"Juliana S Bernardes, Alberto Davila, Vitor San...",A study of structural properties on profiles HMMs,cs.AI,Juliana Bernardes,"6 pages, 7 figures",NaN,NaN,NaN,http://arxiv.org/licenses/nonexclusive-distrib...,Motivation: Profile hidden Markov Models (pH...,2007-04-16,2008-12-11,2007-04-01,CV


## **그래프 그리기**

### **분야별 시간에 따른 논문 발행 수 선 그래프**

In [ ]:
#각 카테고리 별로 연월 별 개수 그룹화
cla_list_name={'is_CV':[is_CV,'CV'],'is_NLP':[is_NLP,'NLP'],'is_RL':[is_RL,'RL'],'is_Au':[is_Au,'Audio'],'is_Se':[is_Se,'Sequential'],'is_Gr':[is_Gr,'Graph']}
cla_list_ym=['is_CV_ym','is_NLP_ym','is_RL_ym','is_Au_ym','is_Se_ym','is_Gr_ym']

for i,j in zip(cla_list_name,cla_list_ym):
  globals()[j]=cla_list_name[i][0][['id','created_datetime_month']].groupby('created_datetime_month').count().rename(columns={'id': 'count'}) #날짜 기준으로 그룹화
  globals()[j]['date']=globals()[j].index #날짜 인덱스를 칼럼으로
  globals()[j]['category']=cla_list_name[i][1] #카테고리 열 생성

In [ ]:
is_CV_ym

,count,date,category
created_datetime_month,,,
2007-04-01,3,2007-04-01,CV
2007-05-01,3,2007-05-01,CV
2007-06-01,1,2007-06-01,CV
2007-07-01,3,2007-07-01,CV
2007-09-01,2,2007-09-01,CV
...,...,...,...
2023-04-01,939,2023-04-01,CV
2023-05-01,1357,2023-05-01,CV
2023-06-01,1070,2023-06-01,CV


In [ ]:
#카테고리 별로 데이터프레임 복제
cla_list_ym={'is_CV_ym':is_CV_ym,'is_NLP_ym':is_NLP_ym,'is_RL_ym':is_RL_ym,'is_Au_ym':is_Au_ym,'is_Se_ym':is_Se_ym,'is_Gr_ym':is_Gr_ym}
cla_list_zero_name=['is_CV_ym_zero','is_NLP_ym_zero','is_RL_ym_zero','is_Au_ym_zero','is_Se_ym_zero','is_Gr_ym_zero']


for i,j in zip(cla_list_ym,cla_list_zero_name):
  globals()[j]=cla_list_ym[i]  #카테고리 별 데이터프레임 복제

In [ ]:
is_Se_ym

In [ ]:
#연월 리스트 만들기
dates=pd.date_range(start='20070401', end='20230801', freq='MS') #첫 달부터 마지막 달까지 리스트 생성 ('MX':달의 첫째날)
date_list=[]
for i in dates:
  if ((i.strftime('%m')=='01')|(i.strftime('%m')=='02')|(i.strftime('%m')=='03')):
    pass
  else :
    date_list.append(i)


In [ ]:
#카테고리 별로 날짜가 없는 경우 값이 0인 행을 넣은 데이터프레임 만들기(1,2,3월은 제외)
cla_list_zero={'is_CV_ym':is_CV_ym_zero,'is_NLP_ym':is_NLP_ym_zero,'is_RL_ym':is_RL_ym_zero,'is_Au_ym':is_Au_ym_zero,'is_Se_ym':is_Se_ym_zero,'is_Gr_ym':is_Gr_ym_zero}


for i in cla_list_zero:
  for j in date_list:
      if (cla_list_zero[i]['date'] == j).any(axis=0):
        pass
      else:
        cla_list_zero[i].loc[j]=(0,j,cla_list_zero[i]['category'][0])


In [ ]:
is_CV_ym_zero

In [ ]:
#me)이 식으로 정렬이 왜 안 되지?
for i in cla_list_ym:
  cla_list_ym[i]=cla_list_ym[i].sort_values('date')

In [ ]:
#날짜 기준 정렬
is_CV_ym=is_CV_ym.sort_values('date')
is_NLP_ym=is_NLP_ym.sort_values('date')
is_RL_ym=is_RL_ym.sort_values('date')
is_Au_ym=is_Au_ym.sort_values('date')
is_Se_ym=is_Se_ym.sort_values('date')
is_Gr_ym=is_Gr_ym.sort_values('date')

In [ ]:
is_Se_ym

In [ ]:
cla_list_ym={'is_CV_ym':is_CV_ym,'is_NLP_ym':is_NLP_ym,'is_RL_ym':is_RL_ym,'is_Au_ym':is_Au_ym,'is_Se_ym':is_Se_ym,'is_Gr_ym':is_Gr_ym}
#하나로 합치기
data_ym_list=[]
for i in cla_list_ym:
  data_ym_list.append(cla_list_ym[i])
data_zero=pd.concat(data_ym_list)
data_zero

,count,date,category
created_datetime_month,,,
2007-04-01,3,2007-04-01,CV
2007-05-01,3,2007-05-01,CV
2007-06-01,1,2007-06-01,CV
2007-07-01,3,2007-07-01,CV
2007-08-01,0,2007-08-01,CV
...,...,...,...
2023-04-01,443,2023-04-01,Graph
2023-05-01,671,2023-05-01,Graph
2023-06-01,499,2023-06-01,Graph


In [ ]:
data_zero.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 894 entries, 2007-04-01 to 2023-08-01
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   count     894 non-null    int64         
 1   date      894 non-null    datetime64[ns]
 2   category  894 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 27.9+ KB


In [ ]:
#선 그래프 그리기
fig = px.line(data_zero, x='date', y='count', color='category', markers=False, title='카테고리 별 시간에 따른 논문 발행 수')


fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()

### **카테고리 별 총 논문 발행 수 비교**

In [ ]:
data_stack=data_zero[['count','category']].groupby('category').sum()
data_stack['category']=data_stack.index

In [ ]:
fig = px.bar(data_stack, x='category', y='count', color='category',text='count', title="분야별 총 논문 발행 수 비교(2007년 이후)")
fig.update_traces(textposition="outside")
fig.update_layout(showlegend=False, plot_bgcolor='white') #범례 삭제, 배경색 설정
fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = '분야별 총 논문 발행 수', auto_open=True)

'https://plotly.com/~biniy/14/'

In [ ]:
#연도 별 논문 발행 수
cla_list_ym={'is_CV_ym':is_CV_ym,'is_NLP_ym':is_NLP_ym,'is_RL_ym':is_RL_ym,'is_Au_ym':is_Au_ym,'is_Se_ym':is_Se_ym,'is_Gr_ym':is_Gr_ym}

for i in cla_list_ym:
  cla_list_ym[i]['date_y']=cla_list_ym[i]['date'].apply(lambda x: x.strftime('%Y')) #연도 칼럼 생성


NameError: ignored

In [ ]:
is_CV_ym

NameError: ignored

In [ ]:
y_name=['is_CV_y','is_NLP_y','is_RL_y','is_Au_y','is_Se_y','is_Gr_y']

#연도 별 그룹화
for i,j in zip(cla_list_ym,y_name):
  globals()[j]=cla_list_ym[i][['date_y','count']].groupby('date_y').sum()

In [ ]:
y_val={'is_CV_y':is_CV_y,'is_NLP_y':is_NLP_y,'is_RL_y':is_RL_y,'is_Au_y':is_Au_y,'is_Se_y':is_Se_y,'is_Gr_y':is_Gr_y}
cla_list_name={'is_CV':[is_CV,'CV'],'is_NLP':[is_NLP,'NLP'],'is_RL':[is_RL,'RL'],'is_Au':[is_Au,'Audio'],'is_Se':[is_Se,'Sequential'],'is_Gr':[is_Gr,'Graph']}

#인덱스를 칼럼으로
for i,j in zip(y_val,cla_list_name):
  y_val[i]['category']=cla_list_name[j][1]

In [ ]:
is_CV_y

In [ ]:
#하나로 합치기
data_y_list=[]
for i in y_val:
  data_y_list.append(y_val[i])
data_y=pd.concat(data_y_list)
data_y['date_y']=data_y.index
data_y

In [ ]:
#연도 별 그룹화
data_y=data_y.reset_index(drop=True)
data_y_sum=data_y[['date_y','count']].groupby('date_y').sum()
data_y_sum['date_y']=data_y_sum.index
data_y_sum

In [ ]:
fig = px.bar(data_y, x='date_y', y='count',color='category', title="연도 별 논문 발행 수")
fig.update_traces(textposition="outside")
fig.add_trace(go.Scatter(x=data_y_sum['date_y'], y=data_y_sum['count']+400,mode='text',text=data_y_sum['count'], showlegend=False)) #총 논문 수 뿌려주기
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
))
fig.show()

###**월 별 / 카테고리 별 논문 발행 수 그래프**

In [ ]:
data_zero

,count,date,category
created_datetime_month,,,
2007-04-01,3,2007-04-01,CV
2007-05-01,3,2007-05-01,CV
2007-06-01,1,2007-06-01,CV
2007-07-01,3,2007-07-01,CV
2007-08-01,0,2007-08-01,CV
...,...,...,...
2023-04-01,443,2023-04-01,Graph
2023-05-01,671,2023-05-01,Graph
2023-06-01,499,2023-06-01,Graph


In [ ]:
is_CV_ym

,count,date,category,date_y
created_datetime_month,,,,
2007-04-01,3,2007-04-01,CV,2007
2007-05-01,3,2007-05-01,CV,2007
2007-06-01,1,2007-06-01,CV,2007
2007-07-01,3,2007-07-01,CV,2007
2007-09-01,2,2007-09-01,CV,2007
...,...,...,...,...
2023-04-01,890,2023-04-01,CV,2023
2023-05-01,1244,2023-05-01,CV,2023
2023-06-01,976,2023-06-01,CV,2023


In [ ]:
#날짜를 숫자로 바꿔주기(그래야 graph obj가 인식)
data_zero['date_int']=pd.to_numeric(data_zero['date'].apply(lambda x: x.strftime('%Y%m%d')[0:-2]))

In [ ]:
data_zero

In [ ]:
data_zero['count'].max() #y축 최대값

2620

In [ ]:
data_zero['date_str']=[str(i)[:4]+'.'+str(i)[-2:] for i in data_zero['date_int']]

In [ ]:
fig = px.bar(data_zero, x="category", y="count", animation_frame="date_int", animation_group="category",text='count',
            color="category", hover_name="category",range_y=(0,data_zero['count'].max()),
           title='연월 별 각 카테고리의 논문 발행 수')
fig.update_layout(sliders=[{"currentvalue": {"prefix": "Date="}}])
fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = '연도 별 각 카테고리의 논문 발행 수', auto_open=True)

'https://plotly.com/~biniy/61/'

###**연도 별 / 카테고리 별 논문 발행 수**

In [ ]:
is_CV_ym

,count,date,category,create_y
created_datetime_month,,,,
2007-04-01,3,2007-04-01,CV,2007
2007-05-01,3,2007-05-01,CV,2007
2007-06-01,1,2007-06-01,CV,2007
2007-07-01,3,2007-07-01,CV,2007
2007-08-01,0,2007-08-01,CV,2007
...,...,...,...,...
2023-04-01,939,2023-04-01,CV,2023
2023-05-01,1357,2023-05-01,CV,2023
2023-06-01,1070,2023-06-01,CV,2023


In [ ]:
cla_list_ym={'is_CV_ym':is_CV_ym,'is_NLP_ym':is_NLP_ym,'is_RL_ym':is_RL_ym,'is_Au_ym':is_Au_ym,'is_Se_ym':is_Se_ym,'is_Gr_ym':is_Gr_ym}
#연도 열 추가
for i in cla_list_ym:
  cla_list_ym[i]['create_y']=cla_list_ym[i]['date'].dt.strftime('%Y')

In [ ]:
#연도별 그룹화
cla_list_y=['is_CV_y','is_NLP_y','is_RL_y','is_Au_y','is_Se_y','is_Gr_y']

for i,j in zip(cla_list_ym,cla_list_y):
  globals()[j]=cla_list_ym[i][['count','create_y']].groupby('create_y').sum()

In [ ]:
#연도,카테고리 열 추가
cla_list_name={'is_CV':[is_CV,'CV'],'is_NLP':[is_NLP,'NLP'],'is_RL':[is_RL,'RL'],'is_Au':[is_Au,'Audio'],'is_Se':[is_Se,'Sequential'],'is_Gr':[is_Gr,'Graph']}
cla_y={'is_CV_y':is_CV_y,'is_NLP_y':is_NLP_y,'is_RL_y':is_RL_y,'is_Au_y':is_Au_y,'is_Se_y':is_Se_y,'is_Gr_y':is_Gr_y}

for i,j in zip(cla_y,cla_list_name):
  cla_y[i]['create_y']=cla_y[i].index
  cla_y[i]['category']=cla_list_name[j][1]

In [ ]:
is_CV_y

In [ ]:
#하나로 합치기
data_y_list=[]
for i in cla_y:
  data_y_list.append(cla_y[i])
data_y=pd.concat(data_y_list)
data_y

In [ ]:
fig = px.bar(data_y, x="category", y="count", animation_frame="create_y", animation_group="category",text='count',
            color="category", hover_name="category",range_y=(0,data_y['count'].max()),
           title='연도 별 각 카테고리의 논문 발행 수')

fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()

## **워드 클라우드(제목 이용)**

In [ ]:
from collections import Counter #개수 세기 모듈
import nltk #자연어 처리 패키지
from nltk.corpus import stopwords #불용어 모듈
nltk.download('all') #nltk 기능 다운로드 (1분)

###**데이터 확인**

In [ ]:
import nltk #자연어 처리 패키지
from nltk.corpus import stopwords #불용어 모듈

In [ ]:
is_CV.head(1)

,ori_index,id,authors,title,categories,submitter,comments,journal-ref,doi,report-no,license,abstract,created_datetime,last_updated_datetime,created_datetime_month,category
10,2009,704.201,"Juliana S Bernardes, Alberto Davila, Vitor San...",A study of structural properties on profiles HMMs,cs.AI,Juliana Bernardes,"6 pages, 7 figures",NaN,NaN,NaN,http://arxiv.org/licenses/nonexclusive-distrib...,Motivation: Profile hidden Markov Models (pH...,2007-04-16,2008-12-11,2007-04-01,CV


**nltk 기능 다운**

In [ ]:
nltk.download('all') #nltk 기능 다운로드 (1분)

### **카테고리 별(전체)**

In [ ]:
#제목만 가져와서 데이터프레임 만들기(카테고리별)
cla_list_name={'is_CV':[is_CV,'CV'],'is_NLP':[is_NLP,'NLP'],'is_RL':[is_RL,'RL'],'is_Au':[is_Au,'Audio'],'is_Se':[is_Se,'Sequential'],'is_Gr':[is_Gr,'Graph']}
titles_list=['title_CV','title_NLP','title_RL','title_Audio','title_Se','title_Gr']

for i,j in zip(cla_list_name,titles_list):
  globals()[j] = cla_list_name[i][0]['title']

In [ ]:
title_CV

10        A study of structural properties on profiles HMMs
35               Robust Multi-Cellular Developmental Design
63        Clustering and Feature Selection using Sparse ...
91        Multi-Sensor Fusion Method using Dynamic Bayes...
98        Autoencoder, Principal Component Analysis and ...
                                ...                        
188364    Unsupervised Learning of Invariance Transforma...
188372    VAPI: Vectorization of Algorithm for Performan...
188375    Evolutionary Multi-objective Optimisation in N...
188380    A Homomorphic Encryption Framework for Privacy...
188382    Noise-Resilient Designs for Optical Neural Net...
Name: title, Length: 37016, dtype: object

####**Tokenizing**

In [ ]:
#각 카테고리별로 토큰화
titles_val={'title_CV':title_CV,'title_NLP':title_NLP,'title_RL':title_RL,'title_Audio':title_Audio,'title_Se':title_Se,'title_Gr':title_Gr}
token_list=['token_CV','token_NLP','token_RL','token_Audio','token_Se','token_Gr']

for i,j in zip(titles_val,token_list):
  globals()[j]= [nltk.word_tokenize(k) for k in titles_val[i]] #제목을 토큰화

In [ ]:
token_CV

In [ ]:
#하나의 리스트로
token_val={'token_CV':token_CV,'token_NLP':token_NLP,'token_RL':token_RL,'token_Audio':token_Audio,'token_Se':token_Se,'token_Gr':token_Gr}
one_list=['one_CV','one_NLP','one_RL','one_Audio','one_Se','one_Gr',]

for i,j in zip(token_val,one_list):
  globals()[j]=[]
  for k in token_val[i]:
    for l in k:
        globals()[j].append(l)

In [ ]:
one_CV

In [ ]:
#소문자로 바꾸기
one_val={'one_CV':one_CV,'one_NLP':one_NLP,'one_RL':one_RL,'one_Audio':one_Audio,'one_Se':one_Se,'one_Gr':one_Gr,}
low_list=['low_CV','low_NLP','low_RL','low_Audio','low_Se','low_Gr']

for i,j in zip(one_val,low_list):
  globals()[j]=[]
  for k in  one_val[i]:
    globals()[j].append(k.lower())


In [ ]:
low_CV

In [ ]:
from nltk.stem import WordNetLemmatizer
low_val={'low_CV':low_CV,'low_NLP':low_NLP,'low_RL':low_RL,'low_Audio':low_Audio,'low_Se':low_Se,'low_Gr':low_Gr}
single_list=['single_CV','single_NLP','single_RL','single_Audio','single_Se','single_Gr']

#명사의 원형 찾기
lm = WordNetLemmatizer()

for i,j in zip(low_val,single_list):
  globals()[j]=[lm.lemmatize(w, pos="n") for w in low_val[i]]

In [ ]:
single_CV

In [ ]:
#nltk 모듈 내장 불용어 리스트
stop_words=stopwords.words('english')
#그 외 불용어 리스트
punc_marks=[',',':','?','(',')','$',"\'s","'",'--','-','&',]
# 두 리스트를 하나로
stops=stop_words+punc_marks

In [ ]:
single_val={'single_CV':single_CV,'single_NLP':single_NLP,'single_RL':single_RL,'single_Audio':single_Audio,'single_Se':single_Se,'single_Gr':single_Gr}
token_fin_list=['token_fin_CV','token_fin_NLP','token_fin_RL','token_fin_Audio','token_fin_Se','token_fin_Gr']

#불용어 제거
for i,j in zip(single_val,token_fin_list):
  globals()[j] = []
  for k in single_val[i]:
    if k not in stops:
        globals()[j].append(k)

In [ ]:
token_fin_Audio

In [ ]:
from nltk.tag import pos_tag

In [ ]:
#품사 붙이기(1분)
token_fin_val={'token_fin_CV':token_fin_CV,'token_fin_NLP':token_fin_NLP,'token_fin_RL':token_fin_RL,'token_fin_Audio':token_fin_Audio,'token_fin_Se':token_fin_Se,'token_fin_Gr':token_fin_Gr}
pos_list=['pos_CV','pos_NLP','pos_RL','pos_Audio','pos_Se','pos_Gr'] #pos: 품사

for i,j in zip(token_fin_val,pos_list):
  globals()[j]=pd.DataFrame(pos_tag(token_fin_val[i]),columns=['word','pos'])

In [ ]:
pos_CV

In [ ]:
pos_val={'pos_CV':pos_CV,'pos_NLP':pos_NLP,'pos_RL':pos_RL,'pos_Audio':pos_Audio,'pos_Se':pos_Se,'pos_Gr':pos_Gr}
#count 열 만들기
for i in pos_val:
  pos_val[i]['count']=1

In [ ]:
pos_val={'pos_CV':pos_CV,'pos_NLP':pos_NLP,'pos_RL':pos_RL,'pos_Audio':pos_Audio,'pos_Se':pos_Se,'pos_Gr':pos_Gr}
count_list=['count_CV','count_NLP','count_RL','count_Audio','count_Se','count_Gr',]

#단어 별 개수 세기
for i,j in zip(pos_val,count_list):
  globals()[j]=pos_val[i].groupby(['pos','word'], as_index = False).count()

In [ ]:
count_CV

In [ ]:
count_val={'count_CV':count_CV,'count_NLP':count_NLP,'count_RL':count_RL,'count_Audio':count_Audio,'count_Se':count_Se,'count_Gr':count_Gr,}
word_count_list=['word_count_CV','word_count_NLP','word_count_RL','word_count_Audio','word_count_Se','word_count_Gr',]

#단어 기준 그룹화
for i,j in zip(count_val,word_count_list):
  globals()[j]=count_val[i][['word','count']].groupby('word',as_index = False).sum()

In [ ]:
word_count_val={'word_count_CV':word_count_CV,'word_count_NLP':word_count_NLP,'word_count_RL':word_count_RL,'word_count_Audio':word_count_Audio,'word_count_Se':word_count_Se,'word_count_Gr':word_count_Gr}
top_list=['top_CV','top_NLP','top_RL','top_Audio','top_Se','top_Gr']
for i,j in zip(word_count_val,top_list):
  globals()[j]=word_count_val[i].sort_values(by='count', ascending=False).head(100)

In [ ]:
top_CV

,Unnamed: 0,word,count
0,16624,network,6919
1,13496,learning,6835
2,16652,neural,4144
3,5977,deep,3743
4,11827,image,3608
...,...,...,...
95,15635,motion,365
96,26371,understanding,361
97,12203,information,356
98,1918,augmentation,355


In [ ]:
top_val={'top_CV':top_CV,'top_NLP':top_NLP,'top_RL':top_RL,'top_Audio':top_Audio,'top_Se':top_Se,'top_Gr':top_Gr,}
pos_total_list=['pos_total_CV','pos_total_NLP','pos_total_RL','pos_total_Audio','pos_total_Se','pos_total_Gr',]
#품사별로 테이블 만들기
for i,j in zip(top_val,pos_total_list):
  df1=top_val[i][['word','pos']].groupby('pos',).count()
  df2=top_val[i][['word','pos']].groupby('pos',).agg(lambda x: ','.join(set(x)))
  globals()[j]=pd.merge(df1,df2,on='pos')
  globals()[j].to_csv(f'/content/drive/MyDrive/Colab Notebooks/{j}.csv')

In [ ]:
pos_total_NLP

In [ ]:
count_val={'count_CV':count_CV,'count_NLP':count_NLP,'count_RL':count_RL,'count_Audio':count_Audio,'count_Se':count_Se,'count_Gr':count_Gr,}

In [ ]:
pos_val={'pos_CV':pos_CV,'pos_NLP':pos_NLP,'pos_RL':pos_RL,'pos_Audio':pos_Audio,'pos_Se':pos_Se,'pos_Gr':pos_Gr}
pos_count_list=['pos_count_CV','pos_count_NLP','pos_count_RL','pos_count_Audio','pos_count_Se','pos_count_Gr']

#품사별로 테이블 만들기. 이거 아님
for i,j in zip(pos_val,pos_count_list):
  df1=pos_val[i].groupby('pos',).count()
  df2=pos_val[i].groupby('pos',).agg(lambda x: ','.join(set(x)))
  globals()[j]=pd.merge(df1,df2,on='pos')
  #globals()[j].to_csv(f'/content/drive/MyDrive/Colab Notebooks/{i}.csv')

In [ ]:
pos_count_CV

NameError: ignored

In [ ]:
count_val={'count_CV':count_CV,'count_NLP':count_NLP,'count_RL':count_RL,'count_Audio':count_Audio,'count_Se':count_Se,'count_Gr':count_Gr,}
pos_val={'pos_CV':pos_CV,'pos_NLP':pos_NLP,'pos_RL':pos_RL,'pos_Audio':pos_Audio,'pos_Se':pos_Se,'pos_Gr':pos_Gr}
pos_count_list=['pos_count_CV','pos_count_NLP','pos_count_RL','pos_count_Audio','pos_count_Se','pos_count_Gr']
for i,j in zip(pos_val,pos_count_list):
  df1=pos_val[i][['pos','count']].groupby('pos', as_index=False).count()
  df2=pos_val[i][['word','pos']].groupby(['pos'], as_index=False).agg(lambda x: ','.join(set(x)))
  globals()[j]=pd.merge(df1,df2,on='pos')
  globals()[j].to_csv(f'/content/drive/MyDrive/Colab Notebooks/{i}.csv')

In [ ]:
top_val={'top_CV':top_CV,'top_NLP':top_NLP,'top_RL':top_RL,'top_Audio':top_Audio,'top_Se':top_Se,'top_Gr':top_Gr,}

#상위 100개 각 카테고리별로 csv 저장
for i in top_val:
  top_val[i].to_csv(f'/content/drive/MyDrive/Colab Notebooks/{i}.csv')

In [ ]:
pos_count_CV

In [ ]:
#개수 세기. 이거 안 씀
for i,j in zip(pos_val,count_list):
  counter=Counter(token_fin_val[i])
  globals()[j]=counter.most_common(100)

#### **word cloud**

In [ ]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import random

In [ ]:
top_list=['top_CV','top_NLP','top_RL','top_Audio','top_Se','top_Gr',]
#파일 불러오기

for i in top_list:
  file_list=sorted(glob(f'/content/drive/MyDrive/Colab Notebooks/{i}.csv'))
  globals()[i]=pd.DataFrame()
  for file in tqdm(file_list):
    print(file_list)
    temp = pd.read_csv(file, encoding='utf-8')
    globals()[i] = pd.concat([globals()[i], temp], axis=0)

  0%|          | 0/1 [00:00<?, ?it/s]

['/content/drive/MyDrive/Colab Notebooks/top_CV.csv']


  0%|          | 0/1 [00:00<?, ?it/s]

['/content/drive/MyDrive/Colab Notebooks/top_NLP.csv']


  0%|          | 0/1 [00:00<?, ?it/s]

...

['/content/drive/MyDrive/Colab Notebooks/top_Audio.csv']


  0%|          | 0/1 [00:00<?, ?it/s]

['/content/drive/MyDrive/Colab Notebooks/top_Se.csv']


  0%|          | 0/1 [00:00<?, ?it/s]

['/content/drive/MyDrive/Colab Notebooks/top_Gr.csv']


In [ ]:
top_val={'top_CV':top_CV,'top_NLP':top_NLP,'top_RL':top_RL,'top_Audio':top_Audio,'top_Se':top_Se,'top_Gr':top_Gr,}

In [ ]:
len([i for i in range(5,995,25)])

40

In [ ]:
#제일 큰 글자부터 차례로 30개
x0=[500,680,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,850,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(5,110,7)]
x32=[i for i in range(900,1000,7)]
y3=[i for i in range(0,1000,34)]
##그 다음 40개(100)
x4=[i for i in range(5,995,25)] #20개

y4=[i for i in range(5,995,25)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x4)
random.shuffle(y4)

x=x0+x31+x32+x4
y=y0+y3+y4

In [ ]:
top_val={'top_CV':top_CV,'top_NLP':top_NLP,'top_RL':top_RL,'top_Audio':top_Audio,'top_Se':top_Se,'top_Gr':top_Gr,}
color_code={'CV':[20,5,40],'NLP':[65,60,70],'RL':[140,130,170],'Audio':[200,185,210],'Sequential':[260,220,270],'Graph':[340,310,350]}
name_list=['CV','NLP','RL','Audio','Sequential','Graph']
sizing={}
for i,j in zip(top_val,name_list):
  sizing[j]=170/top_val[i]['count'][0]

In [ ]:
sizing

{'CV': 0.028905911258852435,
 'NLP': 0.014622020763269484,
 'RL': 0.11723329425556858,
 'Audio': 3.1746031746031744,
 'Sequential': 0.21551724137931033,
 'Graph': 0.05105948429920858}

In [ ]:
#그래프 그리는 함수 정의
def go_scatter(datas,sizing,c0,cs,ce): #c0,cs,ce:컬러 범위 지정
  #컬러 지정-hsv컬러를 랜덤하게 생성
  def ran_per(a,b): return random.randrange(a,b)
  def ran_hue(a,b): return random.randrange(a,b)
  color_list=[f'hsv({c0},70%,800%)']+[f'hsv({ran_hue(cs,ce)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(datas))]

  #hover 지정
  hovertext=[]
  for i,j in zip(datas['word'],datas['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=x,
          y=y,
          textfont = {'size':np.array(datas['count'])*sizing,
                          'color':color_list},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas['word'],
      )
  )

#도화지 생성
fig = go.Figure()

#그래프 그리기
top_val={'top_CV':top_CV,'top_NLP':top_NLP,'top_RL':top_RL,'top_Audio':top_Audio,'top_Se':top_Se,'top_Gr':top_Gr,}
for i,j,k in zip(top_val,sizing,color_code):
  go_scatter(top_val[i],sizing[j],color_code[k][0],color_code[k][1],color_code[k][2])


#버튼 하나씩 만들기
name_dict={'CV':'Computer Vision','NLP':'Natural Language Processing','RL':'Reinforcement Learning',
               'Audio':'Audio','Sequential':'Sequential','Graph':'Graph'}
buttons=[]
for i,j in zip(range(len(fig.data)),name_dict):
  button=dict(
      label=j,
      method='update',
      args=[{'visible':[True if k==i else False for k in range(len(fig.data))]},
          {'title':name_dict[j]}]
  )
  buttons.append(button)

# 드롭다운 버튼 생성
fig.update_layout(
    updatemenus=[
        dict(
            type='dropdown',
            buttons=buttons,
            direction="up",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=1,
            xanchor="right",
            y=-0.1,
            yanchor="bottom",
        ),
    ]
)
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    plot_bgcolor='white'
                  )
fig.show()



In [ ]:
pip install chart_studio

In [ ]:
import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'word cloud', auto_open=True)

'https://plotly.com/~biniy/47/'

### **카테고리 별/연도 별**

#### Tokenize

In [ ]:
is_CV.head(1)

,ori_index,id,authors,title,categories,submitter,comments,journal-ref,doi,report-no,license,abstract,created_datetime,last_updated_datetime,created_datetime_month,category
10,2009,704.201,"Juliana S Bernardes, Alberto Davila, Vitor San...",A study of structural properties on profiles HMMs,cs.AI,Juliana Bernardes,"6 pages, 7 figures",NaN,NaN,NaN,http://arxiv.org/licenses/nonexclusive-distrib...,Motivation: Profile hidden Markov Models (pH...,2007-04-16,2008-12-11,2007-04-01,CV


In [ ]:
is_CV.info()

In [ ]:
cla_list_name={'is_CV':[is_CV,'CV'],'is_NLP':[is_NLP,'NLP'],'is_RL':[is_RL,'RL'],'is_Au':[is_Au,'Audio'],'is_Se':[is_Se,'Sequential'],'is_Gr':[is_Gr,'Graph']}

#연도 칼럼 생성
for i in cla_list_name:
  cla_list_name[i][0]['date_y']=cla_list_name[i][0]['created_datetime_month'].apply(lambda x: x.strftime('%Y'))

In [ ]:
is_NLP.head(1)

,ori_index,id,authors,title,categories,submitter,comments,journal-ref,doi,report-no,license,abstract,created_datetime,last_updated_datetime,created_datetime_month,category,date_y,token
3,984,704.0985,"Mohd Abubakr, R.M.Vinay",Architecture for Pseudo Acausal Evolvable Embe...,cs.NE cs.AI,Mohd Abubakr,"4 pages, 2 figures. Submitted to SASO 2007",NaN,NaN,NaN,NaN,Advances in semiconductor technology are con...,2007-04-07,2007-04-07,2007-04-01,NLP,2007,"[Architecture, for, Pseudo, Acausal, Evolvable..."


In [ ]:
type(is_NLP['token'][3][0])

str

In [ ]:
#연도별 그룹화
year_list=['year_CV','year_NLP','year_RL','year_Audio','year_Se','year_Gr']

for i,j in zip(cla_list_name,year_list):
  globals()[j]=cla_list_name[i][0][['title','date_y']].groupby('date_y', as_index = False).agg(','.join) #연도 기준 그룹화

In [ ]:
'A study of structural properties on profiles'.lower()

'a study of structural properties on profiles'

In [ ]:
#소문자화
year_val={'year_CV':year_CV,'year_NLP':year_NLP,'year_RL':year_RL,'year_Audio':year_Audio,'year_Se':year_Se,'year_Gr':year_Gr}

for i in year_val:
  year_val[i]['title']=year_val[i]['title'].apply(lambda x:x.lower())


In [ ]:
#제목 토큰화
year_val={'year_CV':year_CV,'year_NLP':year_NLP,'year_RL':year_RL,'year_Audio':year_Audio,'year_Se':year_Se,'year_Gr':year_Gr}
#token_list=['token_CV','token_NLP','token_RL','token_Audio','token_Se','token_Gr']

for i in year_val:
  year_val[i]['token']=year_val[i]['title'].apply(lambda x:nltk.word_tokenize(x))
 # .replace('[','').replace(']',''

In [ ]:
year_CV

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
#명사의 원형 찾기
lm = WordNetLemmatizer()

for i in year_val:
  year_val[i]['token']=year_val[i]['token'].apply(lambda x:[lm.lemmatize(w, pos="n") for w in x])

In [ ]:
year_CV

In [ ]:
#nltk 모듈 내장 불용어 리스트
stop_words=stopwords.words('english')
#그 외 불용어 리스트
punc_marks=[',',':','?','(',')','$',"\'s","'",'--','-','&',]
# 두 리스트를 하나로
stops=stop_words+punc_marks

In [ ]:
def delete(x):
  list1=[]
  for i in x:
    if i not in stops:
      list1.append(i)
  return list1

In [ ]:
#불용어 제거
for i in year_val:
  year_val[i]['token']=year_val[i]['token'].apply(lambda x:delete(x))

In [ ]:
year_list=['year_CV','year_NLP','year_RL','year_Audio','year_Se','year_Gr']

for i in year_list:
  globals()[i].to_csv(f'/content/drive/MyDrive/Colab Notebooks/{i}.csv')

In [ ]:
year_CV

#### 파일 불러오기

In [ ]:
year_list=['year_CV','year_NLP','year_RL','year_Audio','year_Se','year_Gr']
#파일 불러오기

for i in year_list:
  file_list=sorted(glob(f'/content/drive/MyDrive/Colab Notebooks/{i}.csv'))
  globals()[i]=pd.DataFrame()
  for file in tqdm(file_list):
    print(file_list)
    temp = pd.read_csv(file, encoding='utf-8')
    globals()[i] = pd.concat([globals()[i], temp], axis=0)

  0%|          | 0/1 [00:00<?, ?it/s]

['/content/drive/MyDrive/Colab Notebooks/year_CV.csv']


  0%|          | 0/1 [00:00<?, ?it/s]

['/content/drive/MyDrive/Colab Notebooks/year_NLP.csv']


  0%|          | 0/1 [00:00<?, ?it/s]

...

['/content/drive/MyDrive/Colab Notebooks/year_Audio.csv']


  0%|          | 0/1 [00:00<?, ?it/s]

['/content/drive/MyDrive/Colab Notebooks/year_Se.csv']


  0%|          | 0/1 [00:00<?, ?it/s]

['/content/drive/MyDrive/Colab Notebooks/year_Gr.csv']


#### CV

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_CV['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

,word,count,year
0,using,5,2007
1,network,4,2007
2,dynamic,3,2007
3,design,2,2007
4,clustering,2,2007
...,...,...,...
95,link,1,2007
96,topology,1,2007
97,distributed,1,2007
98,consensus,1,2007


In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2007

In [ ]:
CV_word=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
descend=CV_word.sort_values('count',ascending=False)['word'].unique()

In [ ]:
CV_word=CV_word.sort_values('count',ascending=False)

In [ ]:
CV_word

In [ ]:
word_info

,word,x,y
0,learning,500,500
1,network,750,300
2,model,260,700
3,neural,300,350
4,image,600,700
...,...,...,...
335,arithmetic,548,682
336,order-statistics,777,783
337,minimax,590,922
338,weighted,309,506


In [ ]:
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,850,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(5,110,7)]
x32=[i for i in range(900,1000,7)]
y3=[i for i in range(0,1000,34)]
##그 다음 40개(100)
x41=[i for i in range(5,995,25)] #20개
#x42=[i for i in range(600,1000,22)] #20개
y4=[i for i in range(5,995,25)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(0,1000),len(descend)-100)
ys=random.sample(range(0,1000),len(descend)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
word_info=pd.DataFrame(data=descend ,columns=['word'] )
word_info['x']=x
word_info['y']=y

In [ ]:
#컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(20,70%,800%)']+[f'hsv({ran_hue(5,40)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]

In [ ]:
word_info['color']=color_list

In [ ]:
graph=pd.merge(CV_word,word_info,on='word')
graph

,word,count,year,x,y,color
0,learning,1381,2022,500,500,"hsv(20,70%,800%)"
1,learning,1293,2021,500,500,"hsv(20,70%,800%)"
2,learning,1103,2020,500,500,"hsv(20,70%,800%)"
3,learning,883,2019,500,500,"hsv(20,70%,800%)"
4,learning,786,2023,500,500,"hsv(20,70%,800%)"
...,...,...,...,...,...,...
1440,arithmetic,2,2010,575,517,"hsv(18,86%,77%)"
1441,order-statistics,2,2010,921,498,"hsv(8,67%,61%)"
1442,minimax,2,2010,214,936,"hsv(23,39%,53%)"
1443,weighted,2,2010,73,931,"hsv(33,64%,70%)"


In [ ]:
def go_scatter(datas,year,sizing):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['count'])*sizing,
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()
'''
go_scatter(graph,2007,0.5)
go_scatter(graph,2008,0.5)
go_scatter(graph,2009,0.5)
go_scatter(graph,2010,0.5)
go_scatter(graph,2011,0.5)
go_scatter(graph,2012,0.5)
go_scatter(graph,2013,0.5)
go_scatter(graph,2014,0.5)
go_scatter(graph,2015,0.2)
'''
go_scatter(graph,2016,0.1)
go_scatter(graph,2017,0.1)
go_scatter(graph,2018,0.1)
go_scatter(graph,2019,0.1)
go_scatter(graph,2020,0.1)
go_scatter(graph,2021,0.1)
go_scatter(graph,2022,0.1)
go_scatter(graph,2023,0.1)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Computer Vision " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000])
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'CV word cloud', auto_open=True)

'https://plotly.com/~biniy/10/'

####NLP

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_NLP['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2007

In [ ]:
NLP_word=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
descend=NLP_word.sort_values('count',ascending=False)['word'].unique()

In [ ]:
NLP_word=NLP_word.sort_values('count',ascending=False)

In [ ]:
NLP_word

In [ ]:
word_info

In [ ]:
#좌표 지정
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,850,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(5,110,7)]
x32=[i for i in range(900,1000,7)]
y3=[i for i in range(0,1000,34)]
##그 다음 40개(100)
x41=[i for i in range(5,995,25)] #20개
#x42=[i for i in range(600,1000,22)] #20개
y4=[i for i in range(5,995,25)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(0,1000),len(descend)-100)
ys=random.sample(range(0,1000),len(descend)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
word_info=pd.DataFrame(data=descend ,columns=['word'] )
word_info['x']=x
word_info['y']=y

In [ ]:
#컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(65,70%,800%)']+[f'hsv({ran_hue(60,70)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]

In [ ]:
word_info['color']=color_list

In [ ]:
graph=pd.merge(NLP_word,word_info,on='word')
graph

,word,count,year,x,y,color
0,learning,2845,2022,500,500,"hsv(65,70%,800%)"
1,learning,2556,2021,500,500,"hsv(65,70%,800%)"
2,learning,2260,2020,500,500,"hsv(65,70%,800%)"
3,learning,1682,2019,500,500,"hsv(65,70%,800%)"
4,learning,1602,2023,500,500,"hsv(65,70%,800%)"
...,...,...,...,...,...,...
1659,block,2,2008,851,127,"hsv(60,21%,69%)"
1660,noise,2,2008,915,860,"hsv(68,24%,63%)"
1661,contour,2,2008,121,296,"hsv(68,66%,69%)"
1662,region-based,2,2008,472,838,"hsv(68,24%,53%)"


In [ ]:
def go_scatter(datas,year,sizing):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['count'])*sizing,
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()
'''
go_scatter(graph,2007,0.5)
go_scatter(graph,2008,0.5)
go_scatter(graph,2009,0.5)
go_scatter(graph,2010,0.5)
go_scatter(graph,2011,0.5)
go_scatter(graph,2012,0.5)
go_scatter(graph,2013,0.5)
go_scatter(graph,2014,0.5)
go_scatter(graph,2015,0.2)
'''
go_scatter(graph,2016,0.1)
go_scatter(graph,2017,0.1)
go_scatter(graph,2018,0.1)
go_scatter(graph,2019,0.1)
go_scatter(graph,2020,0.1)
go_scatter(graph,2021,0.1)
go_scatter(graph,2022,0.1)
go_scatter(graph,2023,0.1)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Natural Language Processing " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000])
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'NLP word cloud', auto_open=True)

'https://plotly.com/~biniy/26/'

#### RL

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_RL['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2007

In [ ]:
RL_word=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
descend=RL_word.sort_values('count',ascending=False)['word'].unique()

In [ ]:
RL_word=RL_word.sort_values('count',ascending=False)

In [ ]:
#좌표 지정
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,850,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(5,110,7)]
x32=[i for i in range(900,1000,7)]
y3=[i for i in range(0,1000,34)]
##그 다음 40개(100)
x41=[i for i in range(5,995,25)] #20개
#x42=[i for i in range(600,1000,22)] #20개
y4=[i for i in range(5,995,25)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(0,1000),len(descend)-100)
ys=random.sample(range(0,1000),len(descend)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
word_info=pd.DataFrame(data=descend ,columns=['word'] )
word_info['x']=x
word_info['y']=y

In [ ]:
#컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(140,70%,800%)']+[f'hsv({ran_hue(130,170)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]

In [ ]:
word_info['color']=color_list

In [ ]:
graph=pd.merge(RL_word,word_info,on='word')
graph

,word,count,year,x,y,color
0,learning,339,2022,500,500,"hsv(140,70%,800%)"
1,learning,319,2020,500,500,"hsv(140,70%,800%)"
2,learning,314,2021,500,500,"hsv(140,70%,800%)"
3,learning,227,2019,500,500,"hsv(140,70%,800%)"
4,learning,164,2023,500,500,"hsv(140,70%,800%)"
...,...,...,...,...,...,...
1037,ensemble,2,2014,164,876,"hsv(140,55%,67%)"
1038,broadcasting,2,2014,482,344,"hsv(160,60%,59%)"
1039,vanet,2,2014,386,518,"hsv(142,50%,77%)"
1040,mapping,2,2014,134,210,"hsv(138,92%,70%)"


In [ ]:
def go_scatter(datas,year,sizing):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['count'])*sizing,
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()
'''
go_scatter(graph,2007,0.5)
go_scatter(graph,2008,0.5)
go_scatter(graph,2009,0.5)
go_scatter(graph,2010,0.5)
go_scatter(graph,2011,0.5)
go_scatter(graph,2012,0.5)
go_scatter(graph,2013,0.5)
go_scatter(graph,2014,0.5)
go_scatter(graph,2015,0.2)
'''
go_scatter(graph,2016,0.6)
go_scatter(graph,2017,0.6)
go_scatter(graph,2018,0.6)
go_scatter(graph,2019,0.6)
go_scatter(graph,2020,0.6)
go_scatter(graph,2021,0.6)
go_scatter(graph,2022,0.6)
go_scatter(graph,2023,0.6)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Reinforcement Learning " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000])
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'RL word cloud', auto_open=True)

'https://plotly.com/~biniy/33/'

#### Audio

In [ ]:
year_Audio

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_Audio['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2007

In [ ]:
Audio_word=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
descend=Audio_word.sort_values('count',ascending=False)['word'].unique()

In [ ]:
Audio_word=Audio_word.sort_values('count',ascending=False)

In [ ]:
Audio_word

In [ ]:
word_info

In [ ]:
#좌표 지정
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,850,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(5,110,7)]
x32=[i for i in range(900,1000,7)]
y3=[i for i in range(0,1000,34)]
##그 다음 40개(100)
x41=[i for i in range(5,995,25)] #20개
#x42=[i for i in range(600,1000,22)] #20개
y4=[i for i in range(5,995,25)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(0,1000),len(descend)-100)
ys=random.sample(range(0,1000),len(descend)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
word_info=pd.DataFrame(data=descend ,columns=['word'] )
word_info['x']=x
word_info['y']=y

In [ ]:
#컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(200,70%,800%)']+[f'hsv({ran_hue(185,210)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]

In [ ]:
word_info['color']=color_list

In [ ]:
graph=pd.merge(Audio_word,word_info,on='word')
graph

,word,count,year,x,y,color
0,learning,339,2022,500,500,"hsv(200,70%,800%)"
1,learning,319,2020,500,500,"hsv(200,70%,800%)"
2,learning,314,2021,500,500,"hsv(200,70%,800%)"
3,learning,227,2019,500,500,"hsv(200,70%,800%)"
4,learning,164,2023,500,500,"hsv(200,70%,800%)"
...,...,...,...,...,...,...
876,non-parallel,2,2015,402,341,"hsv(196,57%,68%)"
877,linear,2,2015,766,157,"hsv(190,63%,75%)"
878,data-driven,2,2015,351,217,"hsv(189,19%,63%)"
879,wavernn,2,2015,420,532,"hsv(204,43%,65%)"


In [ ]:
def go_scatter(datas,year,sizing):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['count'])*sizing,
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()
'''
go_scatter(graph,2007,0.5)
go_scatter(graph,2008,0.5)
go_scatter(graph,2009,0.5)
go_scatter(graph,2010,0.5)
go_scatter(graph,2011,0.5)
go_scatter(graph,2012,0.5)
go_scatter(graph,2013,0.5)
go_scatter(graph,2014,0.5)
go_scatter(graph,2015,0.2)
'''
go_scatter(graph,2016,0.8)
go_scatter(graph,2017,0.8)
go_scatter(graph,2018,0.8)
go_scatter(graph,2019,0.8)
go_scatter(graph,2020,0.8)
go_scatter(graph,2021,0.8)
go_scatter(graph,2022,0.8)
go_scatter(graph,2023,0.8)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Audio " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000])
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'Audio word cloud', auto_open=True)

'https://plotly.com/~biniy/35/'

#### Sequential

In [ ]:
year_Se

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_Se['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2007

In [ ]:
Se_word=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
descend=Se_word.sort_values('count',ascending=False)['word'].unique()

In [ ]:
Se_word=Se_word.sort_values('count',ascending=False)

In [ ]:
Se_word

In [ ]:
Se_info

In [ ]:
#좌표 지정
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,850,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(5,110,7)]
x32=[i for i in range(900,1000,7)]
y3=[i for i in range(0,1000,34)]
##그 다음 40개(100)
x41=[i for i in range(5,995,25)] #20개
#x42=[i for i in range(600,1000,22)] #20개
y4=[i for i in range(5,995,25)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(0,1000),len(descend)-100)
ys=random.sample(range(0,1000),len(descend)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
word_info=pd.DataFrame(data=descend ,columns=['word'] )
word_info['x']=x
word_info['y']=y

In [ ]:
#컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(260,70%,800%)']+[f'hsv({ran_hue(220,270)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]

In [ ]:
word_info['color']=color_list

In [ ]:
graph=pd.merge(Se_word,word_info,on='word')
graph

,word,count,year,x,y,color
0,learning,1087,2022,500,500,"hsv(260,70%,800%)"
1,learning,605,2023,500,500,"hsv(260,70%,800%)"
2,learning,145,2019,500,500,"hsv(260,70%,800%)"
3,learning,139,2018,500,500,"hsv(260,70%,800%)"
4,learning,139,2020,500,500,"hsv(260,70%,800%)"
...,...,...,...,...,...,...
1106,answer,2,2013,394,355,"hsv(232,81%,60%)"
1107,speaker,2,2013,943,843,"hsv(223,75%,63%)"
1108,state,2,2013,538,789,"hsv(257,50%,70%)"
1109,tracker,2,2013,695,965,"hsv(245,46%,69%)"


In [ ]:
def go_scatter(datas,year,sizing):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['count'])*sizing,
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()
'''
go_scatter(graph,2007,0.5)
go_scatter(graph,2008,0.5)
go_scatter(graph,2009,0.5)
go_scatter(graph,2010,0.5)
go_scatter(graph,2011,0.5)
go_scatter(graph,2012,0.5)
go_scatter(graph,2013,0.5)
go_scatter(graph,2014,0.5)
go_scatter(graph,2015,0.2)
'''
go_scatter(graph,2016,0.2)
go_scatter(graph,2017,0.2)
go_scatter(graph,2018,0.2)
go_scatter(graph,2019,0.2)
go_scatter(graph,2020,0.2)
go_scatter(graph,2021,0.2)
go_scatter(graph,2022,0.2)
go_scatter(graph,2023,0.2)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Sequential " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000])
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'Sequential word cloud', auto_open=True)

'https://plotly.com/~biniy/43/'

#### Graph

In [ ]:
year_Gr

,date_y,title,token
0,2007,ensemble learning for free with evolutionary a...,"[ensemble, learning, free, evolutionary, algor..."
1,2008,a new approach to automated epileptic diagnosi...,"[new, approach, automated, epileptic, diagnosi..."
2,2009,on maximum a posteriori estimation of hidden m...,"[maximum, posteriori, estimation, hidden, mark..."
3,2010,introducing dendritic cells as a novel immune-...,"[introducing, dendritic, cell, novel, immune-i..."
4,2011,"kernels for global constraints,rapid feature l...","[kernel, global, constraint, rapid, feature, l..."
5,2012,explaining adaptation in genetic algorithms wi...,"[explaining, adaptation, genetic, algorithm, u..."
6,2013,interactive pomdp lite: towards practical plan...,"[interactive, pomdp, lite, towards, practical,..."
7,2014,distribution-aware sampling and weighted model...,"[distribution-aware, sampling, weighted, model..."
8,2015,semantic enrichment of mobile phone data recor...,"[semantic, enrichment, mobile, phone, data, re..."
9,2016,an efficient algorithm for mining frequent seq...,"[efficient, algorithm, mining, frequent, seque..."


In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_Gr['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2007

,word,count,year
0,on-line,3,2007
1,ensemble,2,2007
2,learning,2,2007
3,monitoring,2,2007
4,intelligence,2,2007


In [ ]:
Gr_word=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
descend=Gr_word.sort_values('count',ascending=False)['word'].unique()

In [ ]:
Gr_word=Gr_word.sort_values('count',ascending=False)

In [ ]:
Gr_word

,word,count,year
0,learning,1087,2022
0,learning,910,2021
0,learning,605,2023
0,learning,597,2020
1,network,490,2021
...,...,...,...
67,compression,2,2012
68,point,2,2012
69,function,2,2012
70,parallel,2,2012


In [ ]:
#좌표 지정
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,850,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(5,110,7)]
x32=[i for i in range(900,1000,7)]
y3=[i for i in range(0,1000,34)]
##그 다음 40개(100)
x41=[i for i in range(5,995,25)] #20개
#x42=[i for i in range(600,1000,22)] #20개
y4=[i for i in range(5,995,25)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(0,1000),len(descend)-100)
ys=random.sample(range(0,1000),len(descend)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
word_info=pd.DataFrame(data=descend ,columns=['word'] )
word_info['x']=x
word_info['y']=y

In [ ]:
#컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(340,70%,800%)']+[f'hsv({ran_hue(310,350)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]

In [ ]:
word_info['color']=color_list

In [ ]:
word_info

In [ ]:
graph=pd.merge(Gr_word,word_info,on='word')
graph

In [ ]:
def go_scatter(datas,year,sizing): #그래프 그려주는 함수

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['count'])*sizing,
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()
'''
go_scatter(graph,2007,0.5)
go_scatter(graph,2008,0.5)
go_scatter(graph,2009,0.5)
go_scatter(graph,2010,0.5)
go_scatter(graph,2011,0.5)
go_scatter(graph,2012,0.5)
go_scatter(graph,2013,0.5)
go_scatter(graph,2014,0.5)
go_scatter(graph,2015,0.2)
'''
go_scatter(graph,2016,0.3)
go_scatter(graph,2017,0.3)
go_scatter(graph,2018,0.3)
go_scatter(graph,2019,0.3)
go_scatter(graph,2020,0.3)
go_scatter(graph,2021,0.3)
go_scatter(graph,2022,0.3)
go_scatter(graph,2023,0.3)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Graph " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000])
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'Graph word cloud', auto_open=True)

'https://plotly.com/~biniy/41/'

###**카테고리 별(공통 단어 제외)**

In [ ]:
#개수 세고 데이터프레임화
token_fin_val={'token_fin_CV':token_fin_CV,'token_fin_NLP':token_fin_NLP,'token_fin_RL':token_fin_RL,'token_fin_Audio':token_fin_Audio,'token_fin_Se':token_fin_Se,'token_fin_Gr':token_fin_Gr}
count_list=['count_CV','count_NLP','count_RL','count_Audio','count_Se','count_Gr']

for i,j in zip(token_fin_val,count_list):
  counter=Counter(token_fin_val[i])
  globals()[j]=pd.DataFrame(dict(counter).keys(),columns=['word'])
  globals()[j]['count']=dict(counter).values()

In [ ]:
#개수가 한 개인 단어 삭제
count_list=['count_CV','count_NLP','count_RL','count_Audio','count_Se','count_Gr']
for i in count_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
#모든 카테고리의 단어 모으기
words=pd.concat([count_CV['word'],count_NLP['word'],count_RL['word'],count_Audio['word'],count_Se['word'],count_Gr['word']])

In [ ]:
#중복 제거
words=words.drop_duplicates()

In [ ]:
len(words)

15440

In [ ]:
#제거 전 개수
print(len(count_CV))
print(len(count_NLP))
print(len(count_RL))
print(len(count_Audio))
print(len(count_Se))
print(len(count_Gr))

9183
13686
2677
257
2638
5517


In [ ]:
#6개 카테고리 모두에서 공통인 단어 제거(3분)
count_list=['count_CV','count_NLP','count_RL','count_Audio','count_Se','count_Gr']

for i in words:
  is_six=len(count_CV['word'][count_CV['word']==i])+len(count_NLP['word'][count_NLP['word']==i])+len(count_RL['word'][count_RL['word']==i])+len(count_Audio['word'][count_Audio['word']==i])+len(count_Se['word'][count_Se['word']==i])+len(count_Gr['word'][count_Gr['word']==i])
  if is_six==6:
    for j in count_list:
      globals()[j].drop(globals()[j][globals()[j]['word']==i].index,inplace=True)


In [ ]:
#제거 후 개수
print(len(count_CV))
print(len(count_NLP))
print(len(count_RL))
print(len(count_Audio))
print(len(count_Se))
print(len(count_Gr))

8993
13496
2487
67
2448
5327


In [ ]:
#정렬
count_CV.sort_values('count',ascending=False)
count_NLP.sort_values('count',ascending=False)
count_RL.sort_values('count',ascending=False)
count_Audio.sort_values('count',ascending=False)
count_Se.sort_values('count',ascending=False)
count_Gr.sort_values('count',ascending=False)

In [ ]:
#상위 100개 선택
count_list=['count_CV','count_NLP','count_RL','count_Audio','count_Se','count_Gr']
top_list=['top_CV','top_NLP','top_RL','top_Audio','top_Se','top_Gr']

for i,j in zip(count_list,top_list):
  globals()[j]=globals()[i].head(100)

In [ ]:
len(count_Audio)

67

In [ ]:
top_CV.set_index(keys=[[i for i in range(100)]], inplace=True, drop=True)
top_NLP.set_index(keys=[[i for i in range(100)]], inplace=True, drop=True)
top_RL.set_index(keys=[[i for i in range(100)]], inplace=True, drop=True)
top_Audio.set_index(keys=[[i for i in range(67)]], inplace=True, drop=True)
top_Se.set_index(keys=[[i for i in range(100)]], inplace=True, drop=True)
top_Gr.set_index(keys=[[i for i in range(100)]], inplace=True, drop=True)

In [ ]:
len([i for i in range(29,974,19)])

50

In [ ]:
#제일 큰 글자부터 차례로 50개
x0=[500,600,320,650,300,
    300,730,720,125,300, #10
    300,480,750,400,450,
    700,800,830,200,900, #20
    120,600,400,650,900,
    910,80,800,160,900, #30
    880,920,920,680,120,
    580,900,450,400,770, #40
    920,570,580,930,100,
    700,270,900,80,90,   #50
    700,200,930,920,100,
    210,870,110,80,350,   #60
    795,350,650,720,700,
    230,680,550,120,90,   #70
    800,230,70,100,310,
    900,150,80,950,210,   #80
    70,80,930,480,820,
    310,870,300,300,550,  #90
    210,680,820,210,480,
    560,50,70,310,130,    #100
    ]

y0=[500,300,650,100,200,
    100,170,700,280,350, #10
    800,750,800,900,850,
    850,600,450,900,850, #20
    100,900,950,950,300,
    100,450,530,550,550, #30
    220,490,50,780,190,
    30,150,50,700,900,  #40
    700,830,200,780,730,
    50,270,900,650,390, #50
    430,500,580,410,50,
    960,980,870,950,430, #60
    400,580,400,480,600,
    740,550,920,700,800, #70
    950,50,600,350,860,
    940,150,760,350,650,  #80
    520,570,630,100,30,
    980,350,470,550,380, #90
    610,630,80,440,130,
    460,830,900,30,10 #100
    ]


x=x0
y=y0

In [ ]:
top_val={'top_CV':top_CV,'top_NLP':top_NLP,'top_RL':top_RL,'top_Audio':top_Audio,'top_Se':top_Se,'top_Gr':top_Gr,}
color_code={'CV':[20,5,40],'NLP':[74,55,90],'RL':[140,130,170],'Audio':[200,175,215],'Sequential':[260,230,270],'Graph':[340,310,350]}
name_list=['CV','NLP','RL','Audio','Sequential','Graph']
sizing={}
for i,j in zip(top_val,name_list):
  sizing[j]=150/top_val[i]['count'][0]

In [ ]:
sizing

{'CV': 0.12620638455827765,
 'NLP': 0.06888168557536467,
 'RL': 0.17782426778242677,
 'Audio': 8.095238095238095,
 'Sequential': 0.4644808743169399,
 'Graph': 0.2910958904109589}

In [ ]:
#그래프 그리는 함수 정의
def go_scatter(datas,sizing,c0,cs,ce): #c0,cs,ce:컬러 범위 지정
  #컬러 지정-hsv컬러를 랜덤하게 생성
  def ran_per(a,b): return random.randrange(a,b)
  def ran_hue(a,b): return random.randrange(a,b)
  color_list=[f'hsv({c0},70%,800%)']+[f'hsv({ran_hue(cs,ce)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(datas)-1)]

  #hover 지정
  hovertext=[]
  for i,j in zip(datas['word'],datas['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=x,
          y=y,
          textfont = {'size':np.array(datas['count'])*sizing,
                          'color':color_list},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas['word'],
      )
  )

#도화지 생성
fig = go.Figure()

#그래프 그리기
top_val={'top_CV':top_CV,'top_NLP':top_NLP,'top_RL':top_RL,'top_Audio':top_Audio,'top_Se':top_Se,'top_Gr':top_Gr,}
for i,j,k in zip(top_val,sizing,color_code):
  go_scatter(top_val[i],sizing[j],color_code[k][0],color_code[k][1],color_code[k][2])


#버튼 하나씩 만들기
name_dict={'CV':'Computer Vision','NLP':'Natural Language Processing','RL':'Reinforcement Learning',
               'Audio':'Audio','Sequential':'Sequential','Graph':'Graph'}
buttons=[]
for i,j in zip(range(len(fig.data)),name_dict):
  button=dict(
      label=j,
      method='update',
      args=[{'visible':[True if k==i else False for k in range(len(fig.data))]},
          {'title':name_dict[j]}]
  )
  buttons.append(button)

# 드롭다운 버튼 생성
fig.update_layout(
    updatemenus=[
        dict(
            type='dropdown',
            buttons=buttons,
            direction="up",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=1,
            xanchor="right",
            y=-0.1,
            yanchor="bottom",
        ),
    ]
)
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=False,
                               autorange=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=False,
                              autorange=True,
                              range=[0,1000]),
                    plot_bgcolor='white'
                  )
fig.show()



In [ ]:
pip install chart_studio

In [ ]:
import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='biniy'
api_key='CU0Eq5yLLJTSZIdxtza2'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'word cloud_no duplication', auto_open=True)

'https://plotly.com/~biniy/63/'


###**카테고리 별 / 연도 별 워드 클라우드 (연도 별로 상대적 글자 크기 지정)**

####데이터 불러오기

In [ ]:
year_list=['year_CV','year_NLP','year_RL','year_Audio','year_Se','year_Gr']
#파일 불러오기

for i in year_list:
  file_list=sorted(glob(f'/content/drive/MyDrive/Colab Notebooks/{i}.csv'))
  globals()[i]=pd.DataFrame()
  for file in tqdm(file_list):
    print(file_list)
    temp = pd.read_csv(file, encoding='utf-8')
    globals()[i] = pd.concat([globals()[i], temp], axis=0)

In [ ]:
year_CV

In [ ]:
from ast import literal_eval
year_list=['year_CV','year_NLP','year_RL','year_Audio','year_Se','year_Gr']

#문자열로 저장된 token 값을 리스트로 만들어주기
for i in year_list:
  globals()[i]['token']=globals()[i]['token'].apply(lambda x: literal_eval(x))

In [ ]:
type(year_NLP['token'][0])

list

#### CV

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_CV['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2011

In [ ]:
#하나의 데이터프레임으로
word_sum=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
#정렬
word_sum=word_sum.sort_values('count',ascending=False)

In [ ]:
#unique한 단어만 모으기
unique=word_sum['word'].unique()

In [ ]:
len([i for i in range(20,980,24)])

40

In [ ]:
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,800,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,850,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(100,350,17)] #15개
x32=[i for i in range(650,900,17)] #15개
y3=[i for i in range(50,950,31)] #30개
##그 다음 40개(100)
x41=[i for i in range(50,950,23)] #40개
y4=[i for i in range(20,980,24)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(0,1000),len(unique)-100)
ys=random.sample(range(0,1000),len(unique)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
#단어 중복 제거한 데이터프레임
word_info=pd.DataFrame(data=unique ,columns=['word'] )

#각 단어 별 좌표 지정
word_info['x']=x
word_info['y']=y

In [ ]:
#단어 별 컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(20,70%,80%)']+[f'hsv({ran_hue(5,40)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]
word_info['color']=color_list

In [ ]:
word_info

,word,x,y,color
0,learning,500,500,"hsv(20,70%,80%)"
1,network,750,300,"hsv(23,50%,51%)"
2,model,260,700,"hsv(22,69%,59%)"
3,neural,300,350,"hsv(28,25%,55%)"
4,image,600,700,"hsv(37,86%,62%)"
...,...,...,...,...
335,improvement,188,896,"hsv(24,11%,78%)"
336,non-linear,671,499,"hsv(27,97%,63%)"
337,web,558,487,"hsv(18,16%,71%)"
338,association,580,33,"hsv(22,63%,61%)"


In [ ]:
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']


#연도 별 글자 사이즈 지정(기준:각 연도의 가장 많이 나온 단어의 개수)
for i in year_list:
  sizing=120/globals()[i]['count'][0]
  globals()[i]['size']=globals()[i]['count']*sizing

In [ ]:
#하나의 데이터프레임으로 모으기
raw=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
raw

In [ ]:
#완성 데이터
graph=pd.merge(raw,word_info,on='word')

In [ ]:
graph

In [ ]:
def go_scatter(datas,year):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['size']),
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()

go_scatter(graph,2016)
go_scatter(graph,2017)
go_scatter(graph,2018)
go_scatter(graph,2019)
go_scatter(graph,2020)
go_scatter(graph,2021)
go_scatter(graph,2022)
go_scatter(graph,2023)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Computer Vision " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    plot_bgcolor='white'
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='wtmo'
api_key='E0ZadoF4U5lULE4fNque'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'CV word cloud_resize', auto_open=True)

'https://plotly.com/~wtmo/1/'

#### NLP

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_NLP['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2011

In [ ]:
#하나의 데이터프레임으로
word_sum=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
#정렬
word_sum=word_sum.sort_values('count',ascending=False)

In [ ]:
#unique한 단어만 모으기
unique=word_sum['word'].unique()

In [ ]:
word_sum

,word,count,year
0,learning,2845,2022
0,learning,2556,2021
0,learning,2260,2020
0,learning,1682,2019
0,model,1632,2023
...,...,...,...
78,contour,2,2008
77,region-based,2,2008
76,dynamic,2,2008
75,automaton,2,2008


In [ ]:
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,800,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(100,350,17)] #15개
x32=[i for i in range(650,900,17)] #15개
y3=[i for i in range(50,950,31)] #30개
##그 다음 40개(100)
x41=[i for i in range(50,950,23)] #40개
y4=[i for i in range(20,980,24)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(0,1000),len(unique)-100)
ys=random.sample(range(0,1000),len(unique)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
#단어 중복 제거한 데이터프레임
word_info=pd.DataFrame(data=unique ,columns=['word'] )

#각 단어 별 좌표 지정
word_info['x']=x
word_info['y']=y

In [ ]:
color_code={'CV':[20,5,40],'NLP':[65,60,70],'RL':[140,130,170],'Audio':[200,185,210],'Sequential':[260,220,270],'Graph':[340,310,350]}

#단어 별 컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(65,70%,80%)']+[f'hsv({ran_hue(60,70)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]
word_info['color']=color_list

In [ ]:
word_info

,word,x,y,color
0,learning,500,500,"hsv(65,70%,80%)"
1,model,750,300,"hsv(63,53%,58%)"
2,network,260,700,"hsv(66,15%,66%)"
3,transformer,300,350,"hsv(69,62%,74%)"
4,using,600,700,"hsv(69,91%,66%)"
...,...,...,...,...
334,block,269,296,"hsv(67,51%,57%)"
335,noise,179,47,"hsv(69,72%,51%)"
336,contour,172,360,"hsv(62,46%,71%)"
337,region-based,801,513,"hsv(65,40%,70%)"


In [ ]:
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']


#연도 별 글자 사이즈 지정(기준:각 연도의 가장 많이 나온 단어의 개수)
for i in year_list:
  sizing=120/globals()[i]['count'][0]
  globals()[i]['size']=globals()[i]['count']*sizing

In [ ]:
#하나의 데이터프레임으로 모으기
raw=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
raw

,word,count,year,size
0,using,11,2007,120.000000
1,network,10,2007,109.090909
2,neural,7,2007,76.363636
3,system,6,2007,65.454545
4,learning,6,2007,65.454545
...,...,...,...,...
95,retrieval,93,2023,6.838235
96,evaluating,93,2023,6.838235
97,architecture,92,2023,6.764706
98,adaptation,90,2023,6.617647


In [ ]:
graph=pd.merge(raw,word_info,on='word')

In [ ]:
graph

,word,count,year,size,x,y,color
0,using,11,2007,120.000000,600,700,"hsv(66,4%,77%)"
1,using,6,2008,55.384615,600,700,"hsv(66,4%,77%)"
2,using,21,2009,120.000000,600,700,"hsv(66,4%,77%)"
3,using,36,2010,120.000000,600,700,"hsv(66,4%,77%)"
4,using,41,2011,104.680851,600,700,"hsv(66,4%,77%)"
...,...,...,...,...,...,...,...
1659,exploring,123,2023,9.044118,364,764,"hsv(62,89%,63%)"
1660,llm,115,2023,8.455882,646,259,"hsv(69,54%,72%)"
1661,prompt,109,2023,8.014706,285,769,"hsv(60,84%,62%)"
1662,zero-shot,101,2023,7.426471,62,610,"hsv(63,42%,74%)"


In [ ]:
def go_scatter(datas,year):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['size']),
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()

go_scatter(graph,2016)
go_scatter(graph,2017)
go_scatter(graph,2018)
go_scatter(graph,2019)
go_scatter(graph,2020)
go_scatter(graph,2021)
go_scatter(graph,2022)
go_scatter(graph,2023)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Natural Language Processing " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    plot_bgcolor='white'
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='wtmo'
api_key='E0ZadoF4U5lULE4fNque'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'NLP word cloud_resize', auto_open=True)

'https://plotly.com/~wtmo/7/'

#### RL

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_RL['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2011

In [ ]:
#하나의 데이터프레임으로
word_sum=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
#정렬
word_sum=word_sum.sort_values('count',ascending=False)

In [ ]:
#unique한 단어만 모으기
unique=word_sum['word'].unique()

In [ ]:
word_sum

,word,count,year
0,learning,339,2022
0,learning,319,2020
0,learning,314,2021
0,learning,227,2019
1,reinforcement,181,2022
...,...,...,...
36,vanet,2,2014
34,object,2,2014
33,mapping,2,2014
32,deep,2,2014


In [ ]:
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,800,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(100,350,17)] #15개
x32=[i for i in range(650,900,17)] #15개
y3=[i for i in range(50,950,31)] #30개
##그 다음 40개(100)
x41=[i for i in range(50,950,23)] #40개
y4=[i for i in range(20,980,24)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(0,1000),len(unique)-100)
ys=random.sample(range(0,1000),len(unique)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
#단어 중복 제거한 데이터프레임
word_info=pd.DataFrame(data=unique ,columns=['word'] )

#각 단어 별 좌표 지정
word_info['x']=x
word_info['y']=y

In [ ]:
color_code={'CV':[20,5,40],'NLP':[65,60,70],'RL':[140,130,170],'Audio':[200,185,210],'Sequential':[260,220,270],'Graph':[340,310,350]}

#단어 별 컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(140,70%,80%)']+[f'hsv({ran_hue(130,170)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]
word_info['color']=color_list

In [ ]:
word_info

,word,x,y
0,learning,500,500
1,reinforcement,750,300
2,deep,260,700
3,model,300,350
4,network,600,700
...,...,...,...
311,ensemble,967,34
312,broadcasting,610,809
313,vanet,214,258
314,mapping,560,768


In [ ]:
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']


#연도 별 글자 사이즈 지정(기준:각 연도의 가장 많이 나온 단어의 개수)
for i in year_list:
  sizing=120/globals()[i]['count'][0]
  globals()[i]['size']=globals()[i]['count']*sizing

In [ ]:
#하나의 데이터프레임으로 모으기
raw=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
raw

,word,count,year,size
0,using,5,2007,120.000000
1,algorithm,3,2007,72.000000
2,hv,2,2007,48.000000
3,bushing,2,2007,48.000000
4,condition,2,2007,48.000000
...,...,...,...,...
95,out-of-distribution,6,2023,4.390244
96,interpretation,6,2023,4.390244
97,human,6,2023,4.390244
98,diffusion,6,2023,4.390244


In [ ]:
#그래프 데이터
graph=pd.merge(raw,word_info,on='word')

In [ ]:
graph

,word,count,year,size,x,y,color
0,using,5,2007,120.000000,400,210,"hsv(155,9%,52%)"
1,using,3,2009,120.000000,400,210,"hsv(155,9%,52%)"
2,using,3,2010,72.000000,400,210,"hsv(155,9%,52%)"
3,using,4,2011,53.333333,400,210,"hsv(155,9%,52%)"
4,using,11,2012,110.000000,400,210,"hsv(155,9%,52%)"
...,...,...,...,...,...,...,...
1037,augmentation,8,2023,5.853659,594,722,"hsv(169,61%,61%)"
1038,out-of-distribution,6,2023,4.390244,871,430,"hsv(135,35%,73%)"
1039,interpretation,6,2023,4.390244,99,904,"hsv(145,69%,79%)"
1040,diffusion,6,2023,4.390244,236,995,"hsv(168,17%,50%)"


In [ ]:
def go_scatter(datas,year):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['size']),
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()

go_scatter(graph,2016)
go_scatter(graph,2017)
go_scatter(graph,2018)
go_scatter(graph,2019)
go_scatter(graph,2020)
go_scatter(graph,2021)
go_scatter(graph,2022)
go_scatter(graph,2023)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Reinforcement Learning " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    plot_bgcolor='white'
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='wtmo'
api_key='E0ZadoF4U5lULE4fNque'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'RL word cloud_resize', auto_open=True)

'https://plotly.com/~wtmo/10/'

#### Audio

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_Audio['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
year_Audio

,Unnamed: 0,date_y,title,token
0,0,2013,performance analysis of neural network models ...,"[performance, analysis, neural, network, model..."
1,1,2014,ensembles of random sphere cover classifiers,"[ensemble, random, sphere, cover, classifier]"
2,2,2015,real-time audio-to-score alignment of music pe...,"[real-time, audio-to-score, alignment, music, ..."
3,3,2016,"wavenet: a generative model for raw audio,fast...","[wavenet, generative, model, raw, audio, fast,..."
4,4,2017,neural audio synthesis of musical notes with w...,"[neural, audio, synthesis, musical, note, wave..."
5,5,2018,"a universal music translation network,clarinet...","[universal, music, translation, network, clari..."
6,6,2019,multi-reference tacotron by intercross trainin...,"[multi-reference, tacotron, intercross, traini..."
7,7,2020,"latent vector recovery of audio gans,melglow: ...","[latent, vector, recovery, audio, gans, melglo..."
8,8,2021,interpreting intermediate convolutional layers...,"[interpreting, intermediate, convolutional, la..."
9,9,2022,cmelgan: an efficient conditional generative m...,"[cmelgan, efficient, conditional, generative, ..."


In [ ]:
y_2017

In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2011

In [ ]:
#하나의 데이터프레임으로
word_sum=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
#정렬
word_sum=word_sum.sort_values('count',ascending=False)

In [ ]:
#unique한 단어만 모으기
unique=word_sum['word'].unique()

In [ ]:
word_sum

In [ ]:
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    200,600,350,650,100,
    210,80,950,130,800]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,800,
    300,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(100,350,17)] #15개
x32=[i for i in range(650,900,17)] #15개
y3=[i for i in range(50,950,31)] #30개
##그 다음 40개(100)
x41=[i for i in range(50,950,23)] #40개
y4=[i for i in range(20,980,24)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(100,900),len(unique)-100)
ys=random.sample(range(50,950),len(unique)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
#단어 중복 제거한 데이터프레임
word_info=pd.DataFrame(data=unique ,columns=['word'] )

#각 단어 별 좌표 지정
word_info['x']=x
word_info['y']=y

In [ ]:
color_code={'CV':[20,5,40],'NLP':[65,60,70],'RL':[140,130,170],'Audio':[200,185,210],'Sequential':[260,220,270],'Graph':[340,310,350]}

#단어 별 컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(200,70%,80%)']+[f'hsv({ran_hue(185,210)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]
word_info['color']=color_list

In [ ]:
word_info

,word,x,y,color
0,learning,500,500,"hsv(200,70%,80%)"
1,reinforcement,750,300,"hsv(207,20%,63%)"
2,deep,260,700,"hsv(187,49%,67%)"
3,model,300,350,"hsv(198,0%,75%)"
4,network,600,700,"hsv(198,59%,56%)"
...,...,...,...,...
278,non-parallel,703,176,"hsv(198,26%,75%)"
279,linear,210,135,"hsv(187,3%,61%)"
280,data-driven,289,349,"hsv(187,42%,61%)"
281,wavernn,658,113,"hsv(197,32%,75%)"


In [ ]:
year_list=['y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']


#연도 별 글자 사이즈 지정(기준:각 연도의 가장 많이 나온 단어의 개수)
for i in year_list:
  sizing=120/globals()[i]['count'][0]
  globals()[i]['size']=globals()[i]['count']*sizing

In [ ]:
#하나의 데이터프레임으로 모으기
raw=pd.concat([y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
raw

In [ ]:
#그래프 데이터
graph=pd.merge(raw,word_info,on='word')

In [ ]:
graph

In [ ]:
def go_scatter(datas,year):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['size']),
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()

go_scatter(graph,2016)
go_scatter(graph,2017)
go_scatter(graph,2018)
go_scatter(graph,2019)
go_scatter(graph,2020)
go_scatter(graph,2021)
go_scatter(graph,2022)
go_scatter(graph,2023)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Audio " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    plot_bgcolor='white'
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='wtmo'
api_key='E0ZadoF4U5lULE4fNque'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'Audio word cloud_resize', auto_open=True)

'https://plotly.com/~wtmo/15/'

#### Sequential

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_Se['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

,word,count
0,faster,1
1,retrieval,1
2,two-pass,1
3,dynamic-time-warping,1
4,lower,1
5,bound,1


In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2011

,word,count,year
0,model,2,2011
1,neural,2,2011
2,network,2,2011


In [ ]:
#하나의 데이터프레임으로
word_sum=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
#정렬
word_sum=word_sum.sort_values('count',ascending=False)

In [ ]:
#unique한 단어만 모으기
unique=word_sum['word'].unique()

In [ ]:
word_sum

,word,count,year,size
0,learning,1087,2022,120.000000
0,learning,605,2023,120.000000
1,contrastive,465,2022,51.333947
2,network,461,2022,50.892364
3,graph,428,2022,47.249310
...,...,...,...,...
77,tracker,2,2013,NaN
78,fast,2,2013,NaN
79,acoustic,2,2013,NaN
80,bidirectional,2,2013,NaN


In [ ]:
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    400,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,900,
    350,600,350,650,100,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    210,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,800,
    100,900,900,950,790,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(100,350,17)] #15개
x32=[i for i in range(650,900,17)] #15개
y3=[i for i in range(50,950,31)] #30개
##그 다음 40개(100)
x41=[i for i in range(50,950,23)] #40개
y4=[i for i in range(20,980,24)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(0,1000),len(unique)-100)
ys=random.sample(range(0,1000),len(unique)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
#단어 중복 제거한 데이터프레임
word_info=pd.DataFrame(data=unique ,columns=['word'] )

#각 단어 별 좌표 지정
word_info['x']=x
word_info['y']=y

In [ ]:
color_code={'CV':[20,5,40],'NLP':[65,60,70],'RL':[140,130,170],'Audio':[200,185,210],'Sequential':[260,220,270],'Graph':[340,310,350]}

#단어 별 컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(260,70%,80%)']+[f'hsv({ran_hue(220,270)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]
word_info['color']=color_list

In [ ]:
word_info

,word,x,y
0,learning,500,500
1,reinforcement,750,300
2,network,260,700
3,model,300,350
4,neural,600,700
...,...,...,...
284,partially,625,418
285,observable,993,85
286,transition-based,177,199
287,autoencoder,983,195


In [ ]:
year_list=['y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']


#연도 별 글자 사이즈 지정(기준:각 연도의 가장 많이 나온 단어의 개수)
for i in year_list:
  sizing=120/globals()[i]['count'][0]
  globals()[i]['size']=globals()[i]['count']*sizing

In [ ]:
#하나의 데이터프레임으로 모으기
raw=pd.concat([y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
raw

,word,count,year,size
0,model,2,2011,120.000000
1,neural,2,2011,120.000000
2,network,2,2011,120.000000
0,neural,10,2012,120.000000
1,network,7,2012,84.000000
...,...,...,...,...
95,unified,26,2023,5.157025
96,online,26,2023,5.157025
97,similarity,25,2023,4.958678
98,motion,25,2023,4.958678


In [ ]:
#그래프 데이터
graph=pd.merge(raw,word_info,on='word')

In [ ]:
graph

,word,count,year,size,x,y,color
0,model,2,2011,120.000000,400,210,"hsv(261,64%,67%)"
1,model,12,2013,48.000000,400,210,"hsv(261,64%,67%)"
2,model,32,2014,59.076923,400,210,"hsv(261,64%,67%)"
3,model,53,2015,57.818182,400,210,"hsv(261,64%,67%)"
4,model,88,2016,74.893617,400,210,"hsv(261,64%,67%)"
...,...,...,...,...,...,...,...
1105,face,27,2023,5.355372,921,88,"hsv(266,94%,50%)"
1106,multi-view,27,2023,5.355372,913,366,"hsv(238,40%,67%)"
1107,edge,26,2023,5.157025,713,796,"hsv(255,34%,72%)"
1108,context-aware,26,2023,5.157025,884,942,"hsv(256,93%,67%)"


In [ ]:
def go_scatter(datas,year):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['size']),
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()

go_scatter(graph,2016)
go_scatter(graph,2017)
go_scatter(graph,2018)
go_scatter(graph,2019)
go_scatter(graph,2020)
go_scatter(graph,2021)
go_scatter(graph,2022)
go_scatter(graph,2023)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Sequential " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    plot_bgcolor='white'
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='wtmo'
api_key='E0ZadoF4U5lULE4fNque'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'Sequential word cloud_resize', auto_open=True)

'https://plotly.com/~wtmo/25/'

#### Graph

In [ ]:
from collections import Counter #개수 세기 모듈
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

#단어 개수 세기
for i,j in zip(year_Gr['token'],year_list):
  counter=Counter(i)
  globals()[j]=pd.DataFrame(counter.most_common(100),columns=['word','count'])

In [ ]:
y_2007

In [ ]:
#연도 열 추가
for i in year_list:
  globals()[i]['year']=i[2:]

In [ ]:
#개수가 한 개인 단어 삭제
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']

for i in year_list:
  globals()[i]=globals()[i][globals()[i]['count'] != 1]

In [ ]:
y_2011

,word,count,year
0,learning,11,2011
1,model,7,2011
2,system,7,2011
3,using,6,2011
4,agent,5,2011
...,...,...,...
61,opportunistic,2,2011
62,spectrum,2,2011
63,efficient,2,2011
64,regression,2,2011


In [ ]:
#하나의 데이터프레임으로
word_sum=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
#정렬
word_sum=word_sum.sort_values('count',ascending=False)

In [ ]:
#unique한 단어만 모으기
unique=word_sum['word'].unique()

In [ ]:
word_sum

,word,count,year
0,learning,1087,2022
0,learning,910,2021
0,learning,605,2023
0,learning,597,2020
1,network,490,2021
...,...,...,...
67,compression,2,2012
68,point,2,2012
69,function,2,2012
70,parallel,2,2012


In [ ]:
#제일 큰 글자부터 차례로 30개
x0=[500,750,260,300,600,
    100,730,680,260,650,
    450,150,800,260,450,
    700,150,100,200,780,
    350,600,350,650,400,
    210,80,950,130,900]
y0=[500,300,700,350,700,
    790,170,780,150,100,
    780,250,700,800,850,
    850,350,150,900,480,
    100,900,900,950,210,
    90,500,600,400,550]

#그 다음 30개(60)
x31=[i for i in range(100,350,17)] #15개
x32=[i for i in range(650,900,17)] #15개
y3=[i for i in range(50,950,31)] #30개
##그 다음 40개(100)
x41=[i for i in range(50,950,23)] #40개
y4=[i for i in range(20,980,24)] #40개

random.shuffle(x31)
random.shuffle(x32)
random.shuffle(y3)
random.shuffle(x41)
random.shuffle(y4)

xs=random.sample(range(100,900),len(unique)-100)
ys=random.sample(range(20,950),len(unique)-100)

x=x0+x31+x32+x41+xs
y=y0+y3+y4+ys

In [ ]:
#단어 중복 제거한 데이터프레임
word_info=pd.DataFrame(data=unique ,columns=['word'] )

#각 단어 별 좌표 지정
word_info['x']=x
word_info['y']=y

In [ ]:
color_code={'CV':[20,5,40],'NLP':[65,60,70],'RL':[140,130,170],'Audio':[200,185,210],'Sequential':[260,220,270],'Graph':[340,310,350]}

#단어 별 컬러 지정-hsv컬러를 랜덤하게 생성(그룹별로 색상을 다르게 하면 좋을 듯)
def ran_per(a,b): return random.randrange(a,b)
def ran_hue(a,b): return random.randrange(a,b)
color_list=['hsv(340,70%,80%)']+[f'hsv({ran_hue(310,350)},{ran_per(0,100)}%,{ran_per(50,80)}%)' for i in range(len(word_info)-1)]
word_info['color']=color_list

In [ ]:
word_info

,word,x,y,color
0,learning,500,500,"hsv(340,70%,80%)"
1,network,750,300,"hsv(314,23%,69%)"
2,contrastive,260,700,"hsv(325,11%,63%)"
3,graph,300,350,"hsv(349,20%,51%)"
4,neural,600,700,"hsv(349,94%,78%)"
...,...,...,...,...
347,article,504,279,"hsv(339,75%,70%)"
348,cluster,491,714,"hsv(332,34%,50%)"
349,off-line,245,53,"hsv(316,17%,56%)"
350,measurement,768,586,"hsv(314,8%,60%)"


In [ ]:
year_list=['y_2007','y_2008','y_2009','y_2010','y_2011','y_2012','y_2013','y_2014','y_2015','y_2016','y_2017','y_2018','y_2019','y_2020','y_2021','y_2022','y_2023']


#연도 별 글자 사이즈 지정(기준:각 연도의 가장 많이 나온 단어의 개수)
for i in year_list:
  sizing=120/globals()[i]['count'][0]
  globals()[i]['size']=globals()[i]['count']*sizing

In [ ]:
#하나의 데이터프레임으로 모으기
raw=pd.concat([y_2007,y_2008,y_2009,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,y_2021,y_2022,y_2023])

In [ ]:
raw

,word,count,year,size
0,on-line,3,2007,120.000000
1,ensemble,2,2007,80.000000
2,learning,2,2007,80.000000
3,monitoring,2,2007,80.000000
4,intelligence,2,2007,80.000000
...,...,...,...,...
95,unified,26,2023,5.157025
96,online,26,2023,5.157025
97,similarity,25,2023,4.958678
98,motion,25,2023,4.958678


In [ ]:
#그래프 데이터
graph=pd.merge(raw,word_info,on='word')

In [ ]:
graph

,word,count,year,size,x,y,color
0,on-line,3,2007,120.000000,550,671,"hsv(348,32%,64%)"
1,on-line,5,2012,66.666667,550,671,"hsv(348,32%,64%)"
2,on-line,3,2014,30.000000,550,671,"hsv(348,32%,64%)"
3,on-line,3,2015,11.250000,550,671,"hsv(348,32%,64%)"
4,ensemble,2,2007,80.000000,713,163,"hsv(319,3%,79%)"
...,...,...,...,...,...,...,...
1262,forecasting,29,2023,5.752066,673,767,"hsv(346,85%,56%)"
1263,multi-modal,29,2023,5.752066,950,320,"hsv(315,18%,73%)"
1264,multi-view,27,2023,5.355372,469,208,"hsv(344,37%,67%)"
1265,unified,26,2023,5.157025,794,106,"hsv(319,93%,77%)"


In [ ]:
def go_scatter(datas,year):

  hovertext=[]
  for i,j in zip(datas[datas['year']==str(year)]['word'],datas[datas['year']==str(year)]['count']):
    hovertext.append(i+' '+str(j))
  return fig.add_trace(go.Scatter(
          visible=False,
          x=datas[datas['year']==str(year)]['x'],
          y=datas[datas['year']==str(year)]['y'],
          textfont = {'size':np.array(datas[datas['year']==str(year)]['size']),
                          'color':datas[datas['year']==str(year)]['color']},
          hoverinfo='text',
          hovertext=hovertext,
          mode="text",
          text=datas[datas['year']==str(year)]['word'],
      )
  )
fig = go.Figure()

go_scatter(graph,2016)
go_scatter(graph,2017)
go_scatter(graph,2018)
go_scatter(graph,2019)
go_scatter(graph,2020)
go_scatter(graph,2021)
go_scatter(graph,2022)
go_scatter(graph,2023)

# 처음 그래프 생성했을땐 index 0의 Trace 가 보이게 설정
fig.data[0].visible = True

# 스텝 별 슬라이더 데이터 리스트 생성
steps = []
for i in range(len(fig.data)):
    step = dict(
        label =str(i+2016),
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Graph " + str(i+2016)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# 최종 슬라이더 데이터 업데이트
fig.update_layout(
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 1},
    steps=steps
)])
# 축 없애고, 그래프 크기 지정
py.graph_objects.scatter.Marker(opacity=0.5)
fig.update_layout(
                    xaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    yaxis=dict(showgrid=False,
                              showticklabels=False,
                              zeroline=False,
                              automargin=True,
                              range=[0,1000]),
                    plot_bgcolor='white'
                  )

fig.show()

In [ ]:
#pip install chart_studio

import chart_studio.tools #plotly 연동 모듈
import chart_studio.plotly

username='wtmo'
api_key='E0ZadoF4U5lULE4fNque'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

chart_studio.plotly.plot(fig, filename = 'Graph word cloud_resize', auto_open=True)

'https://plotly.com/~wtmo/21/'